# Vietnam - Supplier Loyalty Program

#### Para el armado del siguiente reporte se asumen los siguientes criterios:
##### - Una orden es considerada KO si tiene al menos un producto con manufacturer de KO
##### - Una orden se considera en promoción si tiene al menos un producto en promo 
##### - Se quiere ver info únicamente para los suppliers que se levanten del sheet, no para todos

In [1]:
import fiona
import pandas as pd
import geopandas as gpd
import wabi
from wabi import sql_wabilytics,gSheets,maps_2b
from shapely.geometry import Point, Polygon
import datetime
from datetimerange import DateTimeRange
from datetime import timedelta
import boto3
import io
from gspread_dataframe import set_with_dataframe
import sys
from shapely import wkt
import numpy as np
import time
import shapely.wkt
from shapely.geometry.polygon import Polygon
import simplekml

C:\Users\camilafernandez_wabi\Documents\Notebooks


C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__init__.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__init__.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__init__.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wab

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)


# Countries

In [2]:
countries = ['vn']

# Aux Functions

In [3]:
#function to extract the week number
def week_number(date):    
    week = date.isocalendar()[1]
    return week

#function to extract the year
def extract_year(date):
    year = date.isocalendar()[0]
    return year

#function to extract the date of the start of the week
def GetStartDateFromWeek(year,week):
    iso_date = str(year)+'-W'+str(week)
    start_date_of_week = datetime.datetime.strptime(iso_date + '-1', "%Y-W%W-%w").date()
    return start_date_of_week

In [4]:
#función que recibe una lista de manufacturer y te filtra por los que sean de KO, devolviendo una lista
#con los mismos
def KO_list(manufacturers_list):
    coca_manufacturer_list = []
    for manufacturer in manufacturers_list:
        if manufacturer.find('coca')!=-1 or manufacturer.find('coke')!=-1:
            coca_manufacturer_list.append(manufacturer)
    return coca_manufacturer_list

# Define Scope

In [5]:
#La idea es tener 4 semanas para atrás y la semana en curso
tscope_weeks = 4
tscope_end = datetime.date.today()
tscope_end_str = tscope_end.strftime('%Y-%m-%d')

week = week_number(tscope_end)
year = extract_year(tscope_end)
iso_date = str(year)+'-W'+str(week)
start_date_of_week = GetStartDateFromWeek(year,week)
tscope_start = pd.Timestamp((start_date_of_week+timedelta(weeks=-(tscope_weeks))))
tscope_start_str = tscope_start.strftime('%Y-%m-%d')

In [6]:
weeks_list = []
i = 0
while i<=4:
    week_start = pd.Timestamp((start_date_of_week+timedelta(weeks=-(i))))
    week_start_str = week_start.strftime('%Y-%m-%d')
    weeks_list.append(week_start_str) 
    i+=1

In [7]:
weeks_list

['2022-12-19', '2022-12-12', '2022-12-05', '2022-11-28', '2022-11-21']

# Selected Suppliers

In [8]:
sup_file = gSheets.readSheetByKey('1dcaQxt-xHG_NBuR53LGXar5GV1EtDoEd8qGXHKo6vz0')
sup_tab = sup_file.worksheet('suppliers_id')
suppliers_input_df = pd.DataFrame(sup_tab.get_all_records()).reset_index(drop=True)

In [9]:
suppliers_list = list(set(suppliers_input_df["supplier_id"].astype(str)))
len(suppliers_list)

54

In [10]:
suppliers_query = '''
SELECT 
	s.supplier_id ,
	s.name
FROM wabi2bSuppliers.suppliers s;
'''

In [11]:
suppliers_df_query = sql_wabilytics.querySQL(suppliers_query,'wabi2bSuppliers')

In [12]:
suppliers_df = suppliers_df_query.copy(deep = True)

In [13]:
suppliers_complete_list = list(set(suppliers_df["supplier_id"].astype(str)))
len(suppliers_complete_list)

5132

# Global data

In [14]:
bucket = 'wabi2b-evolution'
s3 = boto3.client('s3')

obj = s3.get_object(Bucket=bucket, Key='global_data.parquet')
global_data0 = pd.read_parquet(io.BytesIO(obj['Body'].read()))

In [15]:
global_data0_reduced = global_data0[['supplier_order_id','order_id','supplier_id', 'status',
                            'supplier','created','confirmed_at','canceled_by','supplier_cancellation_reason',
                            'customer_cancellation_reason','total','discount','city','country_id','created_date','shippedAt']]

In [16]:
#nos quedamos con los países que nos interesan
global_data0_reduced = global_data0_reduced.loc[global_data0_reduced['country_id'].isin(countries)].reset_index(drop=True)

In [17]:
#Me quedo con los suppliers que vienen del input
global_data0_reduced = global_data0_reduced.loc[global_data0_reduced['supplier_id'].isin(suppliers_list)].reset_index(drop=True)

In [18]:
global_data0_reduced = global_data0_reduced[(global_data0_reduced['created_date'] >=tscope_start_str)]

In [19]:
#excluimos las ordenes canceladas
global_data0_reduced = global_data0_reduced[global_data0_reduced['status']=='SHIPPED']

In [20]:
global_data0_reduced

,supplier_order_id,order_id,supplier_id,status,supplier,created,confirmed_at,canceled_by,supplier_cancellation_reason,customer_cancellation_reason,total,discount,city,country_id,created_date,shippedAt
253687,2674050,2482103,340,SHIPPED,Toàn Trường Thịnh,2022-11-21 00:33:05,2022-11-21 00:37:10,None,None,None,2205000.0,33074.84,Ho Chi Minh,vn,2022-11-21,2022-11-21 16:02:17
253688,2674073,2482122,3375,SHIPPED,Đại Lý Anh Thơ,2022-11-21 01:08:36,2022-11-21 01:46:44,None,None,None,6920000.0,102195.00,Ho Chi Minh,vn,2022-11-21,2022-11-21 19:29:47
253689,2674080,2482128,2097,SHIPPED,Nghĩa Phượng,2022-11-21 01:24:40,2022-11-20 22:49:11,None,None,None,6375000.0,95625.00,Da Nang,vn,2022-11-21,2022-11-21 18:03:47
253691,2674111,2482154,2400,SHIPPED,MochiMart,2022-11-21 03:08:45,2022-11-21 01:14:24,None,None,None,5802000.0,87030.00,Da Nang,vn,2022-11-21,2022-11-22 06:11:50
253692,2674162,2482196,2400,SHIPPED,MochiMart,2022-11-21 06:10:38,2022-11-21 07:10:04,None,None,None,2750000.0,55003.44,Da Nang,vn,2022-11-21,2022-11-22 22:14:27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274311,2851136,2648610,1359,SHIPPED,Đại Lý Duy Vinh,2022-12-21 08:51:43,2022-12-21 01:55:32,None,None,None,7271300.0,72713.00,Da Nang,vn,2022-12-21,2022-12-21 10:13:20
274319,2851179,2648651,2785,SHIPPED,Đại lý Anh Quân- Thường Tín,2022-12-21 09:04:38,2022-12-21 03:05:12,None,None,None,10882000.0,100000.00,Hanoi,vn,2022-12-21,2022-12-21 10:59:02
274330,2851221,2648687,2785,SHIPPED,Đại lý Anh Quân- Thường Tín,2022-12-21 09:15:19,2022-12-21 03:05:00,None,None,None,9150000.0,180000.00,Hanoi,vn,2022-12-21,2022-12-21 10:58:22
274395,2851406,2648857,2785,SHIPPED,Đại lý Anh Quân- Thường Tín,2022-12-21 09:55:39,2022-12-21 03:04:10,None,None,None,9941500.0,85895.00,Hanoi,vn,2022-12-21,2022-12-21 11:00:33


# Tab weekly_supplier_ko_prod
#### Para las métricas de esta tab filtramos solamente por productos KO y por los suppliers del input

### Calculating the week_start

In [21]:
global_data0_reduced['year'] = global_data0_reduced['created'].map(extract_year)
global_data0_reduced['week'] = global_data0_reduced['created'].map(week_number)
global_data0_reduced['month'] = pd.DatetimeIndex(global_data0_reduced['created']).month 
global_data0_reduced['week_start'] =global_data0_reduced.apply(lambda x: GetStartDateFromWeek(x['year'],x['week']),axis=1)

In [22]:
global_data0_reduced.head(3)

,supplier_order_id,order_id,supplier_id,status,supplier,created,confirmed_at,canceled_by,supplier_cancellation_reason,customer_cancellation_reason,total,discount,city,country_id,created_date,shippedAt,year,week,month,week_start
253687,2674050,2482103,340,SHIPPED,Toàn Trường Thịnh,2022-11-21 00:33:05,2022-11-21 00:37:10,None,None,None,2205000.0,33074.84,Ho Chi Minh,vn,2022-11-21,2022-11-21 16:02:17,2022,47,11,2022-11-21
253688,2674073,2482122,3375,SHIPPED,Đại Lý Anh Thơ,2022-11-21 01:08:36,2022-11-21 01:46:44,None,None,None,6920000.0,102195.00,Ho Chi Minh,vn,2022-11-21,2022-11-21 19:29:47,2022,47,11,2022-11-21
253689,2674080,2482128,2097,SHIPPED,Nghĩa Phượng,2022-11-21 01:24:40,2022-11-20 22:49:11,None,None,None,6375000.0,95625.00,Da Nang,vn,2022-11-21,2022-11-21 18:03:47,2022,47,11,2022-11-21


In [23]:
KO_Orders = global_data0_reduced.copy(deep = True)

In [24]:
#esto está aperturado por producto y trayendome info únicamente para los suppliers que vienen del input
supplier_orders_items_query = '''
SELECT
	soi.manufacturer,
	soi.supplier_order_id,
	soi.promotion_id,
	so.total,
    soi.supplier_id,
    soi.amount
FROM wabi2bOrders.supplier_order_items soi
LEFT JOIN wabi2bOrders.supplier_order so
ON so.supplier_order_id = soi.supplier_order_id
WHERE soi.supplier_id IN '''+wabi.stringify(suppliers_list)+''';
'''

In [25]:
supplier_order_items_df_query = sql_wabilytics.querySQL(supplier_orders_items_query,'wabi2bOrders')

In [26]:
sup_orders_items_df = supplier_order_items_df_query.copy(deep = True)

In [27]:
sup_orders_items_df[sup_orders_items_df['supplier_order_id']=='1997418']

,manufacturer,supplier_order_id,promotion_id,total,supplier_id,amount
21194,Tân Hiệp Phát,1997418,None,7596000.00,635,850000.0
147469,Nestlé,1997418,None,7596000.00,635,352000.0
195335,Masan Group,1997418,None,7596000.00,635,283000.0
402509,TH Group,1997418,None,7596000.00,635,744000.0
433352,TH Group,1997418,None,7596000.00,635,1155000.0
761425,Masan Group,1997418,None,7596000.00,635,320000.0
801347,TH Group,1997418,None,7596000.00,635,744000.0
1098839,Tân Hiệp Phát,1997418,None,7596000.00,635,356000.0
1122579,PepsiCo,1997418,None,7596000.00,635,546000.0
1239286,Nestlé,1997418,None,7596000.00,635,530000.0


In [28]:
#capaz lo puedo sacar cuando se arregle el bug de la db
#print('before: ', sup_orders_items_df.shape[0])
#sup_orders_items_df = sup_orders_items_df.drop_duplicates().reset_index(drop=True)
#print('after: ', sup_orders_items_df.shape[0])

#### Promotions Orders

In [29]:
#Me quedo con los productos que tienen promo
orders_with_proms = sup_orders_items_df[sup_orders_items_df['promotion_id']==sup_orders_items_df['promotion_id']]

In [30]:
#Lista con las sup orders id que tienen algún producto en promo
orders_with_prom_list = list(set(orders_with_proms["supplier_order_id"].astype(str)))

In [31]:
len(orders_with_prom_list)

3239

In [32]:
#flagueamos las ordenes que tienen algún producto en promo
KO_Orders['order_prom'] = KO_Orders.apply(lambda x: 1 if x['supplier_order_id'] in orders_with_prom_list else 0,axis = 1)

#### KO Orders

In [33]:
#hay algunas ordenes que no tenian manufacturer
sup_orders_items_df = sup_orders_items_df[sup_orders_items_df['manufacturer']==sup_orders_items_df['manufacturer']]

In [34]:
sup_orders_items_df["manufacturer"] = sup_orders_items_df["manufacturer"].apply(lambda x: x.lower())

In [35]:
#aplicamos el set primero para borrar duplicados
manufacturers_list = list(set(sup_orders_items_df["manufacturer"].astype(str)))

In [36]:
coca_manufacturer_list = KO_list(manufacturers_list)

In [37]:
coca_manufacturer_list

['coke marketplace', 'coca cola']

In [38]:
#flagueamos las ordenes que tienen productos de Coca - Cola
sup_orders_items_df['KO_products'] = sup_orders_items_df.apply(lambda x: 1 if x['manufacturer'] in coca_manufacturer_list else 0,axis = 1)

In [39]:
sup_orders_items_df

,manufacturer,supplier_order_id,promotion_id,total,supplier_id,amount,KO_products
0,generico,1424887,None,6691000.00,534,2289000.0,0
1,generico,1588131,None,7499000.00,340,154000.0,0
2,masan group,1921735,None,5404000.00,892,288000.0,0
3,generico,1549099,None,6702000.00,635,152000.0,0
4,masan group,2231756,None,6899000.00,2095,330000.0,0
...,...,...,...,...,...,...,...
1381360,vinamilk,2628262,None,10149000.00,363,990000.0,0
1381361,heineken company,1900539,None,4042000.00,1322,3550000.0,0
1381362,th group,1997418,None,7596000.00,635,1155000.0,0
1381363,interfood,2513793,None,11438000.00,635,142000.0,0


In [40]:
#Nos quedamos con los productos de coca
sup_orders_items_KO_df = sup_orders_items_df[sup_orders_items_df['KO_products']==1]
sup_orders_items_NO_KO_df = sup_orders_items_df[sup_orders_items_df['KO_products']==0]

In [41]:
#acá nos quedamos con las sup orders id que tengan algún producto de KO para poder identificarlas abajo y
#no contar orders repetidas
sup_orders_id_KO_list = list(set(sup_orders_items_KO_df["supplier_order_id"].astype(str)))

In [42]:
KO_Orders['KO_order'] = KO_Orders.apply(lambda x: 1 if x['supplier_order_id'] in sup_orders_id_KO_list else 0,axis = 1)

In [43]:
#nos quedamos únicamente con las ordenes que tienen al menos un producto KO y las ordenes que no son KO
NO_KO_orders = KO_Orders[KO_Orders['KO_order']==0]
KO_Orders = KO_Orders[KO_Orders['KO_order']==1]

In [44]:
KO_Orders_reduced = KO_Orders[['supplier_order_id','order_id','supplier_id','supplier','week_start','city','order_prom']]
NO_KO_Orders_reduced = NO_KO_orders[['supplier_order_id','order_id','supplier_id','supplier','week_start','city']]

In [45]:
#hasta acá tengo identificadas las ordenes con productos de KO y las ordenes con productos de KO que stán en 
#promoción
KO_Orders_reduced

,supplier_order_id,order_id,supplier_id,supplier,week_start,city,order_prom
253692,2674162,2482196,2400,MochiMart,2022-11-21,Da Nang,0
253694,2674163,2482197,635,Loan Ngạn - HBT/Hoàn Kiếm/Hoàng Mai,2022-11-21,Hanoi,0
253696,2674176,2482208,154,Quang Vinh,2022-11-21,Ho Chi Minh,0
253700,2674186,2482217,1783,Ngọc Quang Thành,2022-11-21,Ho Chi Minh,0
253703,2674199,2482228,2375,Đại Lý Bia- Nước Ngọt - Sữa Thanh Huyền,2022-11-21,Ho Chi Minh,0
...,...,...,...,...,...,...,...
274238,2850436,2647949,1063,Đại Lý Thanh Loan,2022-12-19,Ho Chi Minh,0
274241,2850466,2647978,1063,Đại Lý Thanh Loan,2022-12-19,Ho Chi Minh,0
274311,2851136,2648610,1359,Đại Lý Duy Vinh,2022-12-19,Da Nang,0
274319,2851179,2648651,2785,Đại lý Anh Quân- Thường Tín,2022-12-19,Hanoi,0


In [46]:
orders_with_total = supplier_order_items_df_query.copy(deep = True)

In [47]:
orders_with_total = orders_with_total[['supplier_order_id','total']]

In [48]:
orders_with_total[orders_with_total['supplier_order_id']=='1997418']

,supplier_order_id,total
21194,1997418,7596000.00
147469,1997418,7596000.00
195335,1997418,7596000.00
402509,1997418,7596000.00
433352,1997418,7596000.00
761425,1997418,7596000.00
801347,1997418,7596000.00
1098839,1997418,7596000.00
1122579,1997418,7596000.00
1239286,1997418,7596000.00


In [49]:
print('before: ', orders_with_total.shape[0])
orders_with_total = orders_with_total.drop_duplicates().reset_index(drop=True)
print('after: ', orders_with_total.shape[0])

before:  1381365
after:  304903


In [50]:
KO_Orders_with_total = KO_Orders_reduced.merge(orders_with_total, how = 'left', on = 'supplier_order_id').reset_index(drop =True)

In [51]:
KO_Orders_with_total

,supplier_order_id,order_id,supplier_id,supplier,week_start,city,order_prom,total
0,2674162,2482196,2400,MochiMart,2022-11-21,Da Nang,0,2750000.00
1,2674163,2482197,635,Loan Ngạn - HBT/Hoàn Kiếm/Hoàng Mai,2022-11-21,Hanoi,0,8682000.00
2,2674176,2482208,154,Quang Vinh,2022-11-21,Ho Chi Minh,0,11313000.00
3,2674186,2482217,1783,Ngọc Quang Thành,2022-11-21,Ho Chi Minh,0,2698000.00
4,2674199,2482228,2375,Đại Lý Bia- Nước Ngọt - Sữa Thanh Huyền,2022-11-21,Ho Chi Minh,0,10229000.00
...,...,...,...,...,...,...,...,...
8092,2850436,2647949,1063,Đại Lý Thanh Loan,2022-12-19,Ho Chi Minh,0,8673000.00
8093,2850466,2647978,1063,Đại Lý Thanh Loan,2022-12-19,Ho Chi Minh,0,9327000.00
8094,2851136,2648610,1359,Đại Lý Duy Vinh,2022-12-19,Da Nang,0,7271300.00
8095,2851179,2648651,2785,Đại lý Anh Quân- Thường Tín,2022-12-19,Hanoi,0,10882000.00


In [52]:
KO_Orders_with_total['total_promo'] =  KO_Orders_with_total.apply(lambda x: x['total'] if x['order_prom']==1  
                                                                else 0,axis = 1)

In [53]:
KO_Orders_with_total

,supplier_order_id,order_id,supplier_id,supplier,week_start,city,order_prom,total,total_promo
0,2674162,2482196,2400,MochiMart,2022-11-21,Da Nang,0,2750000.00,0
1,2674163,2482197,635,Loan Ngạn - HBT/Hoàn Kiếm/Hoàng Mai,2022-11-21,Hanoi,0,8682000.00,0
2,2674176,2482208,154,Quang Vinh,2022-11-21,Ho Chi Minh,0,11313000.00,0
3,2674186,2482217,1783,Ngọc Quang Thành,2022-11-21,Ho Chi Minh,0,2698000.00,0
4,2674199,2482228,2375,Đại Lý Bia- Nước Ngọt - Sữa Thanh Huyền,2022-11-21,Ho Chi Minh,0,10229000.00,0
...,...,...,...,...,...,...,...,...,...
8092,2850436,2647949,1063,Đại Lý Thanh Loan,2022-12-19,Ho Chi Minh,0,8673000.00,0
8093,2850466,2647978,1063,Đại Lý Thanh Loan,2022-12-19,Ho Chi Minh,0,9327000.00,0
8094,2851136,2648610,1359,Đại Lý Duy Vinh,2022-12-19,Da Nang,0,7271300.00,0
8095,2851179,2648651,2785,Đại lý Anh Quân- Thường Tín,2022-12-19,Hanoi,0,10882000.00,0


In [54]:
KO_Orders_metrics = KO_Orders_with_total.groupby(['supplier_id','supplier','city','week_start']).agg(
Total_KO_Orders = pd.NamedAgg(column='order_id' , aggfunc='nunique'),
Total_KO_Promotion_Orders = pd.NamedAgg(column='order_prom' , aggfunc='sum'),
GMV_KO_promotion_orders = pd.NamedAgg(column='total_promo' , aggfunc='sum')).reset_index()

In [55]:
KO_Orders_metrics[KO_Orders_metrics['supplier_id']=='1038']

,supplier_id,supplier,city,week_start,Total_KO_Orders,Total_KO_Promotion_Orders,GMV_KO_promotion_orders
0,1038,PHAN VŨ,Ho Chi Minh,2022-11-21,116,0,0
1,1038,PHAN VŨ,Ho Chi Minh,2022-11-28,122,1,9623400.00
2,1038,PHAN VŨ,Ho Chi Minh,2022-12-05,135,0,0
3,1038,PHAN VŨ,Ho Chi Minh,2022-12-12,112,1,5830960.00
4,1038,PHAN VŨ,Ho Chi Minh,2022-12-19,32,1,5584960.00


#### GMV KO/ NO KO Product Metric

In [56]:
KO_products = sup_orders_items_KO_df.copy(deep = True)
NO_KO_products = sup_orders_items_NO_KO_df.copy(deep = True)

In [57]:
#estas son las ordenes que tienen al menos algun producto KO y no KO (hicimos todo el proceso arriba)
Orders = KO_Orders_reduced.copy(deep = True)
NO_KO_Orders = NO_KO_Orders_reduced.copy(deep = True)

In [58]:
#hasta acá tengo las ordenes que tienen algún producto de KO
KO_products = KO_products[['supplier_order_id','amount']]
NO_KO_products = NO_KO_products[['supplier_order_id','amount']]

In [59]:
#hacemos merge con los productos para poder tener las ordenes aperturadas por producto KO
KO_products_metric = Orders.merge(KO_products,how= 'left', on = 'supplier_order_id').reset_index()
NO_KO_products_metric = NO_KO_Orders.merge(NO_KO_products,how= 'left', on = 'supplier_order_id').reset_index()

In [60]:
KO_products_metric = KO_products_metric.groupby(['supplier_id','supplier','city','week_start']).agg(
GMV_KO_Prod = pd.NamedAgg(column='amount' , aggfunc='sum')).reset_index()

In [61]:
NO_KO_products_metric = NO_KO_products_metric.groupby(['supplier_id','supplier','city','week_start']).agg(
GMV_NO_KO_Prod = pd.NamedAgg(column='amount' , aggfunc='sum')).reset_index()

In [62]:
KO_Orders_metrics

,supplier_id,supplier,city,week_start,Total_KO_Orders,Total_KO_Promotion_Orders,GMV_KO_promotion_orders
0,1038,PHAN VŨ,Ho Chi Minh,2022-11-21,116,0,0
1,1038,PHAN VŨ,Ho Chi Minh,2022-11-28,122,1,9623400.00
2,1038,PHAN VŨ,Ho Chi Minh,2022-12-05,135,0,0
3,1038,PHAN VŨ,Ho Chi Minh,2022-12-12,112,1,5830960.00
4,1038,PHAN VŨ,Ho Chi Minh,2022-12-19,32,1,5584960.00
...,...,...,...,...,...,...,...
238,892,Đại lý Long Khuê- Thường Tín,Hanoi,2022-12-05,12,0,0
239,892,Đại lý Long Khuê- Thường Tín,Hanoi,2022-12-12,11,0,0
240,892,Đại lý Long Khuê- Thường Tín,Hanoi,2022-12-19,1,0,0
241,968,Đại lý Thùy Trang- Thanh Trì,Hanoi,2022-11-28,1,0,0


In [63]:
#Ahora mergeamos con el df que tiene el resto de las métricas
KO_Metrics = KO_Orders_metrics.merge(KO_products_metric, how='inner',on=['supplier_id','supplier','city','week_start'])

In [64]:
KO_Metrics_complete = KO_Metrics.merge(NO_KO_products_metric, how='left',on=['supplier_id','supplier','city','week_start'])

In [65]:
KO_Metrics_complete.head(3)

,supplier_id,supplier,city,week_start,Total_KO_Orders,Total_KO_Promotion_Orders,GMV_KO_promotion_orders,GMV_KO_Prod,GMV_NO_KO_Prod
0,1038,PHAN VŨ,Ho Chi Minh,2022-11-21,116,0,0,72842000.0,795338180.0
1,1038,PHAN VŨ,Ho Chi Minh,2022-11-28,122,1,9623400.00,83391500.0,688413404.0
2,1038,PHAN VŨ,Ho Chi Minh,2022-12-05,135,0,0,85516500.0,708351620.0


In [66]:
KO_Metrics_complete["week_start"] = KO_Metrics_complete["week_start"].apply(lambda x: x.strftime('%Y-%m-%d'))

### Active KO Products Metrics

In [67]:
orders_reduced = global_data0_reduced.copy(deep = True)

In [68]:
orders_reduced.head(3)

,supplier_order_id,order_id,supplier_id,status,supplier,created,confirmed_at,canceled_by,supplier_cancellation_reason,customer_cancellation_reason,total,discount,city,country_id,created_date,shippedAt,year,week,month,week_start
253687,2674050,2482103,340,SHIPPED,Toàn Trường Thịnh,2022-11-21 00:33:05,2022-11-21 00:37:10,None,None,None,2205000.0,33074.84,Ho Chi Minh,vn,2022-11-21,2022-11-21 16:02:17,2022,47,11,2022-11-21
253688,2674073,2482122,3375,SHIPPED,Đại Lý Anh Thơ,2022-11-21 01:08:36,2022-11-21 01:46:44,None,None,None,6920000.0,102195.00,Ho Chi Minh,vn,2022-11-21,2022-11-21 19:29:47,2022,47,11,2022-11-21
253689,2674080,2482128,2097,SHIPPED,Nghĩa Phượng,2022-11-21 01:24:40,2022-11-20 22:49:11,None,None,None,6375000.0,95625.00,Da Nang,vn,2022-11-21,2022-11-21 18:03:47,2022,47,11,2022-11-21


In [69]:
active_products_query = '''
SELECT
	ap.product_display_id,
	ap.supplier_id,
	ap.price_list,
	pd.manufacturer,
	b.brand_name
FROM wabi2bProducts.active_products ap
LEFT JOIN wabi2bProducts.product_details pd
ON ap.product_display_id = pd.product_display_id
LEFT JOIN wabi2bProducts.brand b 
on b.brand_id = pd.brand_id 
''';

In [70]:
active_products_df = sql_wabilytics.querySQL(active_products_query,'wabi2bProducts')

In [71]:
active_products = active_products_df.copy(deep = True)

In [72]:
active_products

,product_display_id,supplier_id,price_list,manufacturer,brand_name
0,445283,2507,7745,Coca Cola,Kinley
1,535088,5068,12047,โคคา-โคล่า,สไปรท์
2,285414,2485,7695,Coca Cola,Monster
3,357126,3,1451,Generico,VIZZIO
4,195052,1120,7269,Coca Cola,Minute Maid
...,...,...,...,...,...
297147,287609,1655,5743,Groceries Generic,CHESTERFIELD
297148,277474,1963,10106,Genérico,Adega de Pegoes
297149,133734,1485,5365,Generico - Brasil,Garoto
297150,25597,84,12122,COCA-COLA,POWERADE


In [73]:
active_products[(active_products['supplier_id']=='1033') & 
                        (active_products['product_display_id']=='116028') &
                       (active_products['price_list']=='3075')]

,product_display_id,supplier_id,price_list,manufacturer,brand_name
179716,116028,1033,3075,Ting Corporation,Tingco


In [74]:
#filtramos y nos quedamos únicamente con los productos de los suppliers que vienen del input
active_products_reduced = active_products.loc[active_products['supplier_id'].isin(suppliers_list)].reset_index(drop=True)

In [75]:
active_products_reduced

,product_display_id,supplier_id,price_list,manufacturer,brand_name
0,116027,1322,3608,Ting Corporation,Tingco
1,501061,1457,5190,Sabeco Brewery,Saigon Lager
2,138415,968,2944,CALOFIC,Simply
3,140298,2742,8406,Masan Group,Chinsu
4,195180,1359,12219,Saigon Ve Wong,A One
...,...,...,...,...,...
9941,217685,534,1959,Dilmah,Dilmah
9942,527806,433,1739,Habeco,Bia Hà Nội
9943,135675,3190,9341,Afotech,Micoem
9944,97970,1457,5190,Sabeco Brewery,Bia Saigon


In [76]:
#a los productos que vienen de la tabla active products (como es una foto actual) le voy a poner 
active_products_reduced['start_date'] = start_date_of_week.strftime('%Y-%m-%d')
active_products_reduced['end_date'] = datetime.date.today().strftime('%Y-%m-%d')

In [77]:
active_products_reduced

,product_display_id,supplier_id,price_list,manufacturer,brand_name,start_date,end_date
0,116027,1322,3608,Ting Corporation,Tingco,2022-12-19,2022-12-21
1,501061,1457,5190,Sabeco Brewery,Saigon Lager,2022-12-19,2022-12-21
2,138415,968,2944,CALOFIC,Simply,2022-12-19,2022-12-21
3,140298,2742,8406,Masan Group,Chinsu,2022-12-19,2022-12-21
4,195180,1359,12219,Saigon Ve Wong,A One,2022-12-19,2022-12-21
...,...,...,...,...,...,...,...
9941,217685,534,1959,Dilmah,Dilmah,2022-12-19,2022-12-21
9942,527806,433,1739,Habeco,Bia Hà Nội,2022-12-19,2022-12-21
9943,135675,3190,9341,Afotech,Micoem,2022-12-19,2022-12-21
9944,97970,1457,5190,Sabeco Brewery,Bia Saigon,2022-12-19,2022-12-21


In [78]:
active_products_reduced["manufacturer"] = active_products_reduced["manufacturer"].apply(lambda x: x.lower())

In [79]:
manufacturer_list = list(set(active_products_reduced["manufacturer"].astype(str)))

In [80]:
ko_manufacturer_list = KO_list(manufacturer_list)

In [81]:
#nos quedamos únicamente con los productos de cocucha
active_products_reduced = active_products_reduced.loc[active_products_reduced['manufacturer'].isin(ko_manufacturer_list)].reset_index(drop=True)

In [82]:
active_prod = active_products_reduced[['supplier_id','product_display_id','brand_name','price_list','start_date','end_date']]

In [83]:
active_prod

,supplier_id,product_display_id,brand_name,price_list,start_date,end_date
0,363,230428,Coca-Cola,1560,2022-12-19,2022-12-21
1,2406,225829,Coca-Cola,7538,2022-12-19,2022-12-21
2,411,513436,Fanta,1677,2022-12-19,2022-12-21
3,2625,225722,Coca-Cola,8149,2022-12-19,2022-12-21
4,3190,225692,Nutriboost,9341,2022-12-19,2022-12-21
...,...,...,...,...,...,...
689,738,310944,Coca-Cola,2266,2022-12-19,2022-12-21
690,3463,225730,Coca-Cola,9751,2022-12-19,2022-12-21
691,2095,91152,Schweppes,6723,2022-12-19,2022-12-21
692,2134,238172,Fanta,12227,2022-12-19,2022-12-21


#### Nos traemos la info de Active Prod Log

In [84]:
active_products_log_query = '''
select country_id,
supplier_id , 
product_display_id,
status,
start_date,
price_list,
LAG(start_date,1) OVER (PARTITION BY supplier_id,product_display_id,price_list ORDER BY start_date DESC) AS end_date
from
(select s.country_id,
s.supplier_id,
product_display_id,
price_list,
apl.action status,
apl.timestamp start_date
from wabi2bProducts.active_products_log apl 
inner join wabi2bSuppliers.suppliers s on s.supplier_id = apl.supplier_id 
WHERE apl.action in ('DISABLED','ENABLED') 
AND date(apl.`timestamp`) >= "'''+tscope_start_str+'''"
and s.country_id in '''+wabi.stringify(countries)+''') aux 
'''

In [85]:
active_products_log_df = sql_wabilytics.querySQL(active_products_log_query,'wabi2bProducts')

In [86]:
active_products_log = active_products_log_df.copy(deep = True)

In [87]:
active_products_log

,country_id,supplier_id,product_display_id,status,start_date,price_list,end_date
0,vn,1033,116050,DISABLED,2022-12-19 13:32:59,3075,NaT
1,vn,1033,131901,DISABLED,2022-12-07 08:03:47,3075,NaT
2,vn,1033,135645,ENABLED,2022-11-23 13:05:08,3075,NaT
3,vn,1033,190167,ENABLED,2022-11-23 13:01:59,3075,NaT
4,vn,1033,225702,ENABLED,2022-12-01 10:47:08,3075,NaT
...,...,...,...,...,...,...,...
48418,vn,977,66498,ENABLED,2022-12-06 08:25:41,2959,NaT
48419,vn,977,66498,DISABLED,2022-12-05 09:06:48,2959,2022-12-06 08:25:41
48420,vn,977,66499,ENABLED,2022-12-06 08:25:49,2959,NaT
48421,vn,977,66499,DISABLED,2022-12-05 09:06:48,2959,2022-12-06 08:25:49


In [88]:
#nos quedamos con los productos activos solamente
active_products_log = active_products_log[active_products_log['status']=='ENABLED'].reset_index(drop=True)

In [89]:
active_products_log

,country_id,supplier_id,product_display_id,status,start_date,price_list,end_date
0,vn,1033,135645,ENABLED,2022-11-23 13:05:08,3075,NaT
1,vn,1033,190167,ENABLED,2022-11-23 13:01:59,3075,NaT
2,vn,1033,225702,ENABLED,2022-12-01 10:47:08,3075,NaT
3,vn,1033,244003,ENABLED,2022-12-20 10:42:56,3075,NaT
4,vn,1033,389659,ENABLED,2022-12-17 05:40:08,3075,2022-12-21 02:27:53
...,...,...,...,...,...,...,...
14383,vn,977,66393,ENABLED,2022-11-29 02:57:27,2959,2022-12-07 02:47:43
14384,vn,977,66394,ENABLED,2022-12-07 04:43:49,2959,NaT
14385,vn,977,66394,ENABLED,2022-11-29 02:57:29,2959,2022-12-07 02:47:43
14386,vn,977,66498,ENABLED,2022-12-06 08:25:41,2959,NaT


In [90]:
#filtramos por los suppliers que vienen del input
active_products_log_reduced = active_products_log.loc[active_products_log['supplier_id'].isin(suppliers_list)].reset_index(drop=True)

In [91]:
active_products_log_reduced

,country_id,supplier_id,product_display_id,status,start_date,price_list,end_date
0,vn,1038,126114,ENABLED,2022-11-22 04:14:21,3088,NaT
1,vn,1038,135668,ENABLED,2022-12-10 03:34:11,3088,NaT
2,vn,1038,152524,ENABLED,2022-12-10 13:46:53,3088,NaT
3,vn,1038,195031,ENABLED,2022-12-12 04:17:45,3088,NaT
4,vn,1038,218045,ENABLED,2022-12-10 13:47:42,3088,NaT
...,...,...,...,...,...,...,...
3233,vn,892,71940,ENABLED,2022-12-18 06:39:41,2713,NaT
3234,vn,892,97976,ENABLED,2022-12-19 07:06:50,2713,NaT
3235,vn,968,199753,ENABLED,2022-12-06 08:43:02,2944,NaT
3236,vn,968,399982,ENABLED,2022-12-06 08:44:16,2944,NaT


In [92]:
product_display_id_list = set(list(active_products_log_reduced['product_display_id']))

In [93]:
prod_details_query = '''
SELECT
pd.product_display_id,
pd.manufacturer,
b.brand_name 
from wabi2bProducts.product_details pd
left join wabi2bProducts.brand b 
on pd.brand_id = b.brand_id
where pd.product_display_id in '''+wabi.stringify(product_display_id_list)+'''
'''

In [94]:
prod_details_df = sql_wabilytics.querySQL(prod_details_query,'wabi2bProducts')

In [95]:
prod_details = prod_details_df.copy(deep = True)

In [96]:
prod_details

,product_display_id,manufacturer,brand_name
0,102849,Sabeco Brewery,Bia Saigon
1,103499,Ajinomoto,Ajinomoto
2,103501,Ajinomoto,Ajinomoto
3,112456,Heineken Company,Strongbow
4,112457,Heineken Company,Strongbow
...,...,...,...
932,97968,Heineken Company,Larue
933,97970,Sabeco Brewery,Bia Saigon
934,97972,URC Vietnam,C2 Green Tea
935,97974,URC Vietnam,C2 Green Tea


In [97]:
active_products_log_reduced = active_products_log_reduced.merge(prod_details,how='left',on = 'product_display_id')

In [98]:
active_products_log_reduced.head(3)

,country_id,supplier_id,product_display_id,status,start_date,price_list,end_date,manufacturer,brand_name
0,vn,1038,126114,ENABLED,2022-11-22 04:14:21,3088,NaT,URC Vietnam,URC (UNIVERSAL ROBINA CORPORATION)
1,vn,1038,135668,ENABLED,2022-12-10 03:34:11,3088,NaT,Masan Group,Kokomi
2,vn,1038,152524,ENABLED,2022-12-10 13:46:53,3088,NaT,Masan Group,Heo Cao Bồi


In [99]:
active_products_log_reduced["manufacturer"] = active_products_log_reduced["manufacturer"].apply(lambda x: x.lower())

In [100]:
manufacturer_list = list(set(active_products_log_reduced["manufacturer"].astype(str)))

In [101]:
ko_manufacturer_list = KO_list(manufacturer_list)

In [102]:
ko_manufacturer_list

['coke marketplace']

In [103]:
#nos quedamos con los productos de coca
active_products_KO_log_reduced = active_products_log_reduced.loc[active_products_log_reduced['manufacturer'].isin(ko_manufacturer_list)].reset_index(drop=True)

In [104]:
active_products_KO_log_reduced

,country_id,supplier_id,product_display_id,status,start_date,price_list,end_date,manufacturer,brand_name
0,vn,1038,230428,ENABLED,2022-11-24 22:37:27,3088,NaT,coke marketplace,Coca-Cola
1,vn,1038,424553,ENABLED,2022-11-24 14:18:40,3088,NaT,coke marketplace,PREDATOR
2,vn,1038,475301,ENABLED,2022-12-06 01:00:51,3088,NaT,coke marketplace,Coca-Cola
3,vn,1038,513436,ENABLED,2022-11-24 14:17:51,3088,NaT,coke marketplace,Fanta
4,vn,1063,225683,ENABLED,2022-12-03 01:50:09,3149,NaT,coke marketplace,Sprite
...,...,...,...,...,...,...,...,...,...
197,vn,883,513436,ENABLED,2022-12-01 11:31:50,2665,NaT,coke marketplace,Fanta
198,vn,892,157219,ENABLED,2022-12-15 04:27:51,2713,NaT,coke marketplace,Dasani
199,vn,892,225730,ENABLED,2022-12-20 02:21:44,2713,NaT,coke marketplace,Coca-Cola
200,vn,892,230428,ENABLED,2022-12-20 02:25:45,2713,NaT,coke marketplace,Coca-Cola


In [105]:
active_products_KO_log_reduced = active_products_KO_log_reduced[['supplier_id','product_display_id','brand_name','price_list','start_date','end_date']]

In [106]:
active_products_KO_log_reduced

,supplier_id,product_display_id,brand_name,price_list,start_date,end_date
0,1038,230428,Coca-Cola,3088,2022-11-24 22:37:27,NaT
1,1038,424553,PREDATOR,3088,2022-11-24 14:18:40,NaT
2,1038,475301,Coca-Cola,3088,2022-12-06 01:00:51,NaT
3,1038,513436,Fanta,3088,2022-11-24 14:17:51,NaT
4,1063,225683,Sprite,3149,2022-12-03 01:50:09,NaT
...,...,...,...,...,...,...
197,883,513436,Fanta,2665,2022-12-01 11:31:50,NaT
198,892,157219,Dasani,2713,2022-12-15 04:27:51,NaT
199,892,225730,Coca-Cola,2713,2022-12-20 02:21:44,NaT
200,892,230428,Coca-Cola,2713,2022-12-20 02:25:45,NaT


In [107]:
active_products_KO_log_reduced["start_date"] = active_products_KO_log_reduced["start_date"].apply(lambda x: x.strftime('%Y-%m-%d'))

C:\Users\camilafernandez_wabi\AppData\Local\Temp\ipykernel_10844\1354966704.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_products_KO_log_reduced["start_date"] = active_products_KO_log_reduced["start_date"].apply(lambda x: x.strftime('%Y-%m-%d'))


In [108]:
active_products_KO_log_reduced['end_date'] = active_products_KO_log_reduced['end_date'].fillna(datetime.date.today())

C:\Users\camilafernandez_wabi\AppData\Local\Temp\ipykernel_10844\2879301501.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_products_KO_log_reduced['end_date'] = active_products_KO_log_reduced['end_date'].fillna(datetime.date.today())


In [109]:
active_products_KO_log_reduced["end_date"] = active_products_KO_log_reduced["end_date"].apply(lambda x: x.strftime('%Y-%m-%d'))

C:\Users\camilafernandez_wabi\AppData\Local\Temp\ipykernel_10844\3097162416.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_products_KO_log_reduced["end_date"] = active_products_KO_log_reduced["end_date"].apply(lambda x: x.strftime('%Y-%m-%d'))


In [110]:
active_products_KO_log_reduced

,supplier_id,product_display_id,brand_name,price_list,start_date,end_date
0,1038,230428,Coca-Cola,3088,2022-11-24,2022-12-21
1,1038,424553,PREDATOR,3088,2022-11-24,2022-12-21
2,1038,475301,Coca-Cola,3088,2022-12-06,2022-12-21
3,1038,513436,Fanta,3088,2022-11-24,2022-12-21
4,1063,225683,Sprite,3149,2022-12-03,2022-12-21
...,...,...,...,...,...,...
197,883,513436,Fanta,2665,2022-12-01,2022-12-21
198,892,157219,Dasani,2713,2022-12-15,2022-12-21
199,892,225730,Coca-Cola,2713,2022-12-20,2022-12-21
200,892,230428,Coca-Cola,2713,2022-12-20,2022-12-21


In [111]:
#juntamos lo que viene de active products con active products log
products_KO_df = active_products_KO_log_reduced.append(active_prod)

C:\Users\camilafernandez_wabi\AppData\Local\Temp\ipykernel_10844\2890470982.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products_KO_df = active_products_KO_log_reduced.append(active_prod)


In [112]:
products_KO_df

,supplier_id,product_display_id,brand_name,price_list,start_date,end_date
0,1038,230428,Coca-Cola,3088,2022-11-24,2022-12-21
1,1038,424553,PREDATOR,3088,2022-11-24,2022-12-21
2,1038,475301,Coca-Cola,3088,2022-12-06,2022-12-21
3,1038,513436,Fanta,3088,2022-11-24,2022-12-21
4,1063,225683,Sprite,3149,2022-12-03,2022-12-21
...,...,...,...,...,...,...
689,738,310944,Coca-Cola,2266,2022-12-19,2022-12-21
690,3463,225730,Coca-Cola,9751,2022-12-19,2022-12-21
691,2095,91152,Schweppes,6723,2022-12-19,2022-12-21
692,2134,238172,Fanta,12227,2022-12-19,2022-12-21


In [113]:
products_KO_df['range_1'] =products_KO_df.apply(lambda x: DateTimeRange(pd.Timestamp(x['start_date']), pd.Timestamp(x['end_date'])),axis=1)

In [114]:
products_KO_df

,supplier_id,product_display_id,brand_name,price_list,start_date,end_date,range_1
0,1038,230428,Coca-Cola,3088,2022-11-24,2022-12-21,2022-11-24T00:00:00 - 2022-12-21T00:00:00
1,1038,424553,PREDATOR,3088,2022-11-24,2022-12-21,2022-11-24T00:00:00 - 2022-12-21T00:00:00
2,1038,475301,Coca-Cola,3088,2022-12-06,2022-12-21,2022-12-06T00:00:00 - 2022-12-21T00:00:00
3,1038,513436,Fanta,3088,2022-11-24,2022-12-21,2022-11-24T00:00:00 - 2022-12-21T00:00:00
4,1063,225683,Sprite,3149,2022-12-03,2022-12-21,2022-12-03T00:00:00 - 2022-12-21T00:00:00
...,...,...,...,...,...,...,...
689,738,310944,Coca-Cola,2266,2022-12-19,2022-12-21,2022-12-19T00:00:00 - 2022-12-21T00:00:00
690,3463,225730,Coca-Cola,9751,2022-12-19,2022-12-21,2022-12-19T00:00:00 - 2022-12-21T00:00:00
691,2095,91152,Schweppes,6723,2022-12-19,2022-12-21,2022-12-19T00:00:00 - 2022-12-21T00:00:00
692,2134,238172,Fanta,12227,2022-12-19,2022-12-21,2022-12-19T00:00:00 - 2022-12-21T00:00:00


#### Creating the weeks df

In [115]:
weeks_df = pd.DataFrame(columns=('week_start', 'week_end', 'range_2'))

In [116]:
for i in range(0,5):
    start = pd.Timestamp((tscope_start+timedelta(weeks=+(i))))
    start_str = start.strftime('%Y-%m-%d')
    end = pd.Timestamp((start+timedelta(days=6)))
    end_str = end.strftime('%Y-%m-%d')
    #tener en cuenta que este campo es datetimerange.DateTimeRange
    days_range = DateTimeRange(start, end)
    weeks_df.loc[len(weeks_df.index)] = [start_str,end_str,days_range]


In [117]:
weeks_df

,week_start,week_end,range_2
0,2022-11-21,2022-11-27,2022-11-21T00:00:00 - 2022-11-27T00:00:00
1,2022-11-28,2022-12-04,2022-11-28T00:00:00 - 2022-12-04T00:00:00
2,2022-12-05,2022-12-11,2022-12-05T00:00:00 - 2022-12-11T00:00:00
3,2022-12-12,2022-12-18,2022-12-12T00:00:00 - 2022-12-18T00:00:00
4,2022-12-19,2022-12-25,2022-12-19T00:00:00 - 2022-12-25T00:00:00


In [118]:
time_range = DateTimeRange("2022-11-16T00:00:00", "2022-11-18T00:00:00")
x = DateTimeRange("2022-10-17T00:00:00", "2022-10-23T00:00:00")
time_range.is_intersection(x)

False

In [119]:
products_KO_df

,supplier_id,product_display_id,brand_name,price_list,start_date,end_date,range_1
0,1038,230428,Coca-Cola,3088,2022-11-24,2022-12-21,2022-11-24T00:00:00 - 2022-12-21T00:00:00
1,1038,424553,PREDATOR,3088,2022-11-24,2022-12-21,2022-11-24T00:00:00 - 2022-12-21T00:00:00
2,1038,475301,Coca-Cola,3088,2022-12-06,2022-12-21,2022-12-06T00:00:00 - 2022-12-21T00:00:00
3,1038,513436,Fanta,3088,2022-11-24,2022-12-21,2022-11-24T00:00:00 - 2022-12-21T00:00:00
4,1063,225683,Sprite,3149,2022-12-03,2022-12-21,2022-12-03T00:00:00 - 2022-12-21T00:00:00
...,...,...,...,...,...,...,...
689,738,310944,Coca-Cola,2266,2022-12-19,2022-12-21,2022-12-19T00:00:00 - 2022-12-21T00:00:00
690,3463,225730,Coca-Cola,9751,2022-12-19,2022-12-21,2022-12-19T00:00:00 - 2022-12-21T00:00:00
691,2095,91152,Schweppes,6723,2022-12-19,2022-12-21,2022-12-19T00:00:00 - 2022-12-21T00:00:00
692,2134,238172,Fanta,12227,2022-12-19,2022-12-21,2022-12-19T00:00:00 - 2022-12-21T00:00:00


In [120]:
products_KO_df = products_KO_df.merge(weeks_df, how='cross')

In [121]:
products_KO_df

,supplier_id,product_display_id,brand_name,price_list,start_date,end_date,range_1,week_start,week_end,range_2
0,1038,230428,Coca-Cola,3088,2022-11-24,2022-12-21,2022-11-24T00:00:00 - 2022-12-21T00:00:00,2022-11-21,2022-11-27,2022-11-21T00:00:00 - 2022-11-27T00:00:00
1,1038,230428,Coca-Cola,3088,2022-11-24,2022-12-21,2022-11-24T00:00:00 - 2022-12-21T00:00:00,2022-11-28,2022-12-04,2022-11-28T00:00:00 - 2022-12-04T00:00:00
2,1038,230428,Coca-Cola,3088,2022-11-24,2022-12-21,2022-11-24T00:00:00 - 2022-12-21T00:00:00,2022-12-05,2022-12-11,2022-12-05T00:00:00 - 2022-12-11T00:00:00
3,1038,230428,Coca-Cola,3088,2022-11-24,2022-12-21,2022-11-24T00:00:00 - 2022-12-21T00:00:00,2022-12-12,2022-12-18,2022-12-12T00:00:00 - 2022-12-18T00:00:00
4,1038,230428,Coca-Cola,3088,2022-11-24,2022-12-21,2022-11-24T00:00:00 - 2022-12-21T00:00:00,2022-12-19,2022-12-25,2022-12-19T00:00:00 - 2022-12-25T00:00:00
...,...,...,...,...,...,...,...,...,...,...
4475,3375,230428,Coca-Cola,9622,2022-12-19,2022-12-21,2022-12-19T00:00:00 - 2022-12-21T00:00:00,2022-11-21,2022-11-27,2022-11-21T00:00:00 - 2022-11-27T00:00:00
4476,3375,230428,Coca-Cola,9622,2022-12-19,2022-12-21,2022-12-19T00:00:00 - 2022-12-21T00:00:00,2022-11-28,2022-12-04,2022-11-28T00:00:00 - 2022-12-04T00:00:00
4477,3375,230428,Coca-Cola,9622,2022-12-19,2022-12-21,2022-12-19T00:00:00 - 2022-12-21T00:00:00,2022-12-05,2022-12-11,2022-12-05T00:00:00 - 2022-12-11T00:00:00
4478,3375,230428,Coca-Cola,9622,2022-12-19,2022-12-21,2022-12-19T00:00:00 - 2022-12-21T00:00:00,2022-12-12,2022-12-18,2022-12-12T00:00:00 - 2022-12-18T00:00:00


In [122]:
products_KO_df['is_on'] = products_KO_df.apply(lambda x:True 
                                               if x['range_1'].is_intersection(x['range_2']) 
                                               else False,axis=1)

In [123]:
products_KO_on = products_KO_df[products_KO_df['is_on']]

In [124]:
products_KO_on = products_KO_on[['supplier_id','product_display_id','brand_name','price_list','week_start','is_on']]

In [125]:
products_KO_on

,supplier_id,product_display_id,brand_name,price_list,week_start,is_on
0,1038,230428,Coca-Cola,3088,2022-11-21,True
1,1038,230428,Coca-Cola,3088,2022-11-28,True
2,1038,230428,Coca-Cola,3088,2022-12-05,True
3,1038,230428,Coca-Cola,3088,2022-12-12,True
4,1038,230428,Coca-Cola,3088,2022-12-19,True
...,...,...,...,...,...,...
4459,738,310944,Coca-Cola,2266,2022-12-19,True
4464,3463,225730,Coca-Cola,9751,2022-12-19,True
4469,2095,91152,Schweppes,6723,2022-12-19,True
4474,2134,238172,Fanta,12227,2022-12-19,True


In [126]:
print('before: ', products_KO_on.shape[0])
products_KO_on = products_KO_on.drop_duplicates().reset_index(drop=True)
print('after: ', products_KO_on.shape[0])

before:  1312
after:  1124


In [127]:
#le agregamos el nombre al supplier
products_KO_on = products_KO_on.merge(suppliers_df,how='left',on='supplier_id')

In [128]:
products_KO_on

,supplier_id,product_display_id,brand_name,price_list,week_start,is_on,name
0,1038,230428,Coca-Cola,3088,2022-11-21,True,PHAN VŨ
1,1038,230428,Coca-Cola,3088,2022-11-28,True,PHAN VŨ
2,1038,230428,Coca-Cola,3088,2022-12-05,True,PHAN VŨ
3,1038,230428,Coca-Cola,3088,2022-12-12,True,PHAN VŨ
4,1038,230428,Coca-Cola,3088,2022-12-19,True,PHAN VŨ
...,...,...,...,...,...,...,...
1119,1322,225692,Nutriboost,3608,2022-12-19,True,Đại Lý Hoàng
1120,738,310944,Coca-Cola,2266,2022-12-19,True,Đại lý Lâm Nhã- Hà Đông
1121,3463,225730,Coca-Cola,9751,2022-12-19,True,Đại Lý Bội Bội
1122,2095,91152,Schweppes,6723,2022-12-19,True,Nhật Hội


##### Traemos lo poligonos de los suppliers para poder tener la city

In [129]:
suppliers_polygon_query = '''
select DISTINCT
s.supplier_id,
s.country_id
,ST_AsText(sa.polygon) as geometry
,sa.name poly_name
from wabi2bSuppliers.suppliers s 
left join wabi2bSuppliers.suppliers_areas sa on sa.supplier_id = s.supplier_id
where s.country_id in '''+wabi.stringify(countries)+'''
'''

In [130]:
suppliers_polygon_df_query = sql_wabilytics.querySQL(suppliers_polygon_query,'wabi2bSuppliers')

In [131]:
suppliers_polygon_df = suppliers_polygon_df_query.copy(deep = True)

In [132]:
suppliers_polygon_df = suppliers_polygon_df[suppliers_polygon_df['poly_name']==suppliers_polygon_df['poly_name']]

In [133]:
suppliers_polygon_df

,supplier_id,country_id,geometry,poly_name
0,2276,vn,POLYGON((106.68944814460308 10.963493040536603...,Thuận Giao - Thuận An
1,2276,vn,POLYGON((106.66690714283436 10.953961572019447...,An Thạnh - Thuận An
3,1779,vn,POLYGON((106.05902990633885 23.392621204733825...,THANH DUNG
4,2496,vn,"POLYGON((106.6911032267055 10.910216518962008,...",Lái Thiêu - Thuận An
5,2496,vn,POLYGON((106.70511195046655 10.892842102075921...,Bình Hòa - Thuận An
...,...,...,...,...
942,1463,vn,POLYGON((106.51160748047602 10.919076911955463...,Tân Thông Hội
943,1463,vn,POLYGON((106.47961273430599 10.928791651338578...,Tân An Hội
944,1463,vn,POLYGON((106.50312789090965 10.977961472567635...,Phước Vĩnh An
945,2406,vn,POLYGON((108.05660824207337 16.112952150445157...,Đà Nẵng


In [134]:
df_polygons = suppliers_polygon_df.copy(deep=True)
df_polygons["geometry"] = df_polygons["geometry"].astype(str)
df_polygons['geometry'] = df_polygons['geometry'].apply(wkt.loads)
df_polygons['geometry'] = df_polygons['geometry'].apply(lambda x: Polygon(x))

In [135]:
df_polygons = gpd.GeoDataFrame(df_polygons)

In [136]:
s3 = boto3.resource('s3')

obj = s3.Bucket('wabi-ops-zones').Object('wabi2b-zones/wabi2b_zones.kml').get()
full2b_zones_kml= gpd.GeoDataFrame.from_file(obj['Body'], index_col=0)

ciudades_mp= [full2b_zones_kml]

final_cities = gpd.GeoDataFrame()
for ciudad in ciudades_mp:
    final_cities=final_cities.append(ciudad,ignore_index=True)

C:\Users\camilafernandez_wabi\AppData\Local\Temp\ipykernel_10844\3361638333.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_cities=final_cities.append(ciudad,ignore_index=True)


In [137]:
final_cities['Name'].unique()

array(['Buenos Aires', 'Montevideo', 'Cordoba', 'Salta', 'Bogota',
       'Ciudad de Mexico', 'Guadalajara', 'Ho Chi Minh', 'Lima',
       'Medellin', 'San Pablo', 'Santiago de Chile', 'Colima',
       'Rio de Janeiro', 'Puebla', 'San Francisco', 'Pachuca', 'Nairobi',
       'Tijuana', 'Singapur', 'Cairo', 'Salvador', 'Lagos', 'Hanoi',
       'Ciudad Guzman', 'Mérida', 'Tampa', 'Los Ángeles', 'Miami',
       'Mumbai', 'Chennai', 'Delhi', 'Hyderabad', 'Pune', 'Bangalore',
       'Lisboa', 'Moscú', 'Da Nang', 'Tepic', 'North Malaysia',
       'Central Malaysia', 'East Coast Malaysia', 'Alexandria',
       'South Malaysia', 'San Diego', 'Casablanca', 'Mombasa',
       'East Malaysia', 'Laguna', 'Can Tho', 'Pathum Thani'], dtype=object)

In [138]:
df_polygons

,supplier_id,country_id,geometry,poly_name
0,2276,vn,"POLYGON ((106.68945 10.96349, 106.68973 10.963...",Thuận Giao - Thuận An
1,2276,vn,"POLYGON ((106.66691 10.95396, 106.66796 10.955...",An Thạnh - Thuận An
3,1779,vn,"POLYGON ((106.05903 23.39262, 107.02583 22.826...",THANH DUNG
4,2496,vn,"POLYGON ((106.69110 10.91022, 106.69177 10.910...",Lái Thiêu - Thuận An
5,2496,vn,"POLYGON ((106.70511 10.89284, 106.70756 10.895...",Bình Hòa - Thuận An
...,...,...,...,...
942,1463,vn,"POLYGON ((106.51161 10.91908, 106.51839 10.918...",Tân Thông Hội
943,1463,vn,"POLYGON ((106.47961 10.92879, 106.48828 10.924...",Tân An Hội
944,1463,vn,"POLYGON ((106.50313 10.97796, 106.50339 10.977...",Phước Vĩnh An
945,2406,vn,"POLYGON ((108.05661 16.11295, 108.06691 16.169...",Đà Nẵng


In [139]:
### join zones
supplier_cities0 = gpd.overlay(df_polygons, final_cities, how='intersection')
supplier_cities = supplier_cities0[['supplier_id','country_id','Name']].drop_duplicates().reset_index(drop=True)
supplier_cities.rename(columns = {'Name':'city'},inplace = True)
#supplier_cities1 = pd.DataFrame(supplier_cities.groupby([supplier_cities.supplier_id, supplier_cities.country_id]).apply(lambda x: "{%s}" % ', '.join(x.Name))).reset_index()
#supplier_cities1.rename(columns = {0:'cities'},inplace = True)

C:\Users\camilafernandez_wabi\AppData\Local\Temp\ipykernel_10844\2113179115.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  supplier_cities0 = gpd.overlay(df_polygons, final_cities, how='intersection')


In [140]:
products_KO_on_with_cities = products_KO_on.merge(supplier_cities,how='left',on='supplier_id')

In [141]:
products_KO_on_with_cities[(products_KO_on_with_cities['supplier_id']=='1038') & 
                          (products_KO_on_with_cities['city']=='Ho Chi Minh') &
                           (products_KO_on_with_cities['week_start']=='2022-11-21')]

,supplier_id,product_display_id,brand_name,price_list,week_start,is_on,name,country_id,city
0,1038,230428,Coca-Cola,3088,2022-11-21,True,PHAN VŨ,vn,Ho Chi Minh
5,1038,424553,PREDATOR,3088,2022-11-21,True,PHAN VŨ,vn,Ho Chi Minh
13,1038,513436,Fanta,3088,2022-11-21,True,PHAN VŨ,vn,Ho Chi Minh


In [142]:
products_KO_on_with_metric = products_KO_on_with_cities.groupby(['supplier_id','name','city','week_start']).agg(
Active_KO_Brands = pd.NamedAgg(column='brand_name' , aggfunc='nunique')).reset_index()

In [143]:
products_KO_on_with_metric.rename(columns = {'name':'supplier'},inplace = True)

In [144]:
products_KO_on_with_metric

,supplier_id,supplier,city,week_start,Active_KO_Brands
0,1038,PHAN VŨ,Ho Chi Minh,2022-11-21,3
1,1038,PHAN VŨ,Ho Chi Minh,2022-11-28,3
2,1038,PHAN VŨ,Ho Chi Minh,2022-12-05,3
3,1038,PHAN VŨ,Ho Chi Minh,2022-12-12,3
4,1038,PHAN VŨ,Ho Chi Minh,2022-12-19,5
...,...,...,...,...,...
170,892,Đại lý Long Khuê- Thường Tín,Hanoi,2022-11-28,1
171,892,Đại lý Long Khuê- Thường Tín,Hanoi,2022-12-05,1
172,892,Đại lý Long Khuê- Thường Tín,Hanoi,2022-12-12,2
173,892,Đại lý Long Khuê- Thường Tín,Hanoi,2022-12-19,4


In [145]:
KO_Metrics_complete

,supplier_id,supplier,city,week_start,Total_KO_Orders,Total_KO_Promotion_Orders,GMV_KO_promotion_orders,GMV_KO_Prod,GMV_NO_KO_Prod
0,1038,PHAN VŨ,Ho Chi Minh,2022-11-21,116,0,0,72842000.0,795338180.0
1,1038,PHAN VŨ,Ho Chi Minh,2022-11-28,122,1,9623400.00,83391500.0,688413404.0
2,1038,PHAN VŨ,Ho Chi Minh,2022-12-05,135,0,0,85516500.0,708351620.0
3,1038,PHAN VŨ,Ho Chi Minh,2022-12-12,112,1,5830960.00,71437000.0,760856140.0
4,1038,PHAN VŨ,Ho Chi Minh,2022-12-19,32,1,5584960.00,32685500.0,150566580.0
...,...,...,...,...,...,...,...,...,...
238,892,Đại lý Long Khuê- Thường Tín,Hanoi,2022-12-05,12,0,0,14352000.0,91555000.0
239,892,Đại lý Long Khuê- Thường Tín,Hanoi,2022-12-12,11,0,0,19688000.0,331875000.0
240,892,Đại lý Long Khuê- Thường Tín,Hanoi,2022-12-19,1,0,0,920000.0,NaN
241,968,Đại lý Thùy Trang- Thanh Trì,Hanoi,2022-11-28,1,0,0,177000.0,2079000.0


In [146]:
#agregamos la info al df con las otras metricas
KO_Metrics_complete = KO_Metrics_complete.merge(products_KO_on_with_metric, how='left',on=['supplier_id','supplier','city','week_start'])

In [147]:
KO_Metrics_complete.head(3)

,supplier_id,supplier,city,week_start,Total_KO_Orders,Total_KO_Promotion_Orders,GMV_KO_promotion_orders,GMV_KO_Prod,GMV_NO_KO_Prod,Active_KO_Brands
0,1038,PHAN VŨ,Ho Chi Minh,2022-11-21,116,0,0,72842000.0,795338180.0,3.0
1,1038,PHAN VŨ,Ho Chi Minh,2022-11-28,122,1,9623400.00,83391500.0,688413404.0,3.0
2,1038,PHAN VŨ,Ho Chi Minh,2022-12-05,135,0,0,85516500.0,708351620.0,3.0


# Active Promotions
##### Tomamos todas las promos sin importar si es KO o NO KO

In [148]:
#me traigo todas las promos activas desde el scope inicial hasta el dia de hoy, filtrado por vn y también
# para los suppliers del input
promotions_query = '''
select
pi2.promotion_id, 
pi2.supplier_id,
pi2.start_date,
pi2.end_date,
s.name
from wabi2bPromotions.promotion_info pi2
left join wabi2bSuppliers.suppliers s 
on s.supplier_id = pi2.supplier_id 
WHERE pi2.enabled = 1
and pi2.supplier_id in '''+wabi.stringify(suppliers_list)+'''
and date(pi2.start_date) >= "'''+tscope_start_str+'''"
'''

In [149]:
promotions_df_query = sql_wabilytics.querySQL(promotions_query,'wabi2bPromotions')

In [150]:
promotions_df = promotions_df_query.copy(deep = True)

In [151]:
promotions_df["start_date"] = promotions_df["start_date"].apply(lambda x: x.strftime('%Y-%m-%d'))
promotions_df["end_date"] = promotions_df["end_date"].apply(lambda x: x.strftime('%Y-%m-%d'))

In [152]:
promotions_df

,promotion_id,supplier_id,start_date,end_date,name
0,70baf4e3-d2fc-4fea-ae0e-0e817c260ca5,1038,2022-12-19,2022-12-26,PHAN VŨ
1,35bb26c7-5519-489b-af13-4c2fe81d1401,1063,2022-12-19,2022-12-26,Đại Lý Thanh Loan
2,3ffbc2ba-a21b-4bae-a1ee-2dd488117218,1322,2022-12-19,2022-12-26,Đại Lý Hoàng
3,919ef3a7-9efe-469a-98c5-b4dfc036ac7d,1324,2022-12-19,2022-12-26,NGỌC HÙNG
4,4583202c-0309-492d-946e-fbfacf9727d3,1457,2022-12-19,2022-12-26,Hợp Tác Xã P6 Q4
5,2315bcd1-5506-442c-b603-7377105fc408,154,2022-12-19,2022-12-26,Quang Vinh
6,571b8392-55b8-4c11-b7cd-e73fe8fcfa0f,1783,2022-12-18,2022-12-25,Ngọc Quang Thành
7,dfb23240-648f-4785-97b2-a95c58f3f5c4,1783,2022-11-30,2022-12-31,Ngọc Quang Thành
8,5057bb69-7fd9-4adb-a3f4-90dee5970061,1898,2022-12-14,2022-12-31,Hoàng Nhâm
9,7133b898-222a-471d-af07-fc2ce3c5eb2b,1898,2022-12-14,2022-12-31,Hoàng Nhâm


In [153]:
promotions_df['range_1'] =promotions_df.apply(lambda x: DateTimeRange(pd.Timestamp(x['start_date']), pd.Timestamp(x['end_date'])),axis=1)

In [154]:
promotions_df

,promotion_id,supplier_id,start_date,end_date,name,range_1
0,70baf4e3-d2fc-4fea-ae0e-0e817c260ca5,1038,2022-12-19,2022-12-26,PHAN VŨ,2022-12-19T00:00:00 - 2022-12-26T00:00:00
1,35bb26c7-5519-489b-af13-4c2fe81d1401,1063,2022-12-19,2022-12-26,Đại Lý Thanh Loan,2022-12-19T00:00:00 - 2022-12-26T00:00:00
2,3ffbc2ba-a21b-4bae-a1ee-2dd488117218,1322,2022-12-19,2022-12-26,Đại Lý Hoàng,2022-12-19T00:00:00 - 2022-12-26T00:00:00
3,919ef3a7-9efe-469a-98c5-b4dfc036ac7d,1324,2022-12-19,2022-12-26,NGỌC HÙNG,2022-12-19T00:00:00 - 2022-12-26T00:00:00
4,4583202c-0309-492d-946e-fbfacf9727d3,1457,2022-12-19,2022-12-26,Hợp Tác Xã P6 Q4,2022-12-19T00:00:00 - 2022-12-26T00:00:00
5,2315bcd1-5506-442c-b603-7377105fc408,154,2022-12-19,2022-12-26,Quang Vinh,2022-12-19T00:00:00 - 2022-12-26T00:00:00
6,571b8392-55b8-4c11-b7cd-e73fe8fcfa0f,1783,2022-12-18,2022-12-25,Ngọc Quang Thành,2022-12-18T00:00:00 - 2022-12-25T00:00:00
7,dfb23240-648f-4785-97b2-a95c58f3f5c4,1783,2022-11-30,2022-12-31,Ngọc Quang Thành,2022-11-30T00:00:00 - 2022-12-31T00:00:00
8,5057bb69-7fd9-4adb-a3f4-90dee5970061,1898,2022-12-14,2022-12-31,Hoàng Nhâm,2022-12-14T00:00:00 - 2022-12-31T00:00:00
9,7133b898-222a-471d-af07-fc2ce3c5eb2b,1898,2022-12-14,2022-12-31,Hoàng Nhâm,2022-12-14T00:00:00 - 2022-12-31T00:00:00


In [155]:
#usando el week_df que creamos arriba, vemos en que semanas estuvo prendida c/u de las promos
#acá vamos a tener una row por cada week, para después cruzar rangos y ver si estuvo o no prendida la promo
#en esa week
promotions_df_per_week = promotions_df.merge(weeks_df, how='cross')

In [156]:
promotions_df_per_week.head(3)

,promotion_id,supplier_id,start_date,end_date,name,range_1,week_start,week_end,range_2
0,70baf4e3-d2fc-4fea-ae0e-0e817c260ca5,1038,2022-12-19,2022-12-26,PHAN VŨ,2022-12-19T00:00:00 - 2022-12-26T00:00:00,2022-11-21,2022-11-27,2022-11-21T00:00:00 - 2022-11-27T00:00:00
1,70baf4e3-d2fc-4fea-ae0e-0e817c260ca5,1038,2022-12-19,2022-12-26,PHAN VŨ,2022-12-19T00:00:00 - 2022-12-26T00:00:00,2022-11-28,2022-12-04,2022-11-28T00:00:00 - 2022-12-04T00:00:00
2,70baf4e3-d2fc-4fea-ae0e-0e817c260ca5,1038,2022-12-19,2022-12-26,PHAN VŨ,2022-12-19T00:00:00 - 2022-12-26T00:00:00,2022-12-05,2022-12-11,2022-12-05T00:00:00 - 2022-12-11T00:00:00


In [157]:
#identifico cuales fueron las promos prendidas en c/u de las semanas para quedarnos solo con las prendidas
promotions_df_per_week['is_on'] = promotions_df_per_week.apply(lambda x:True 
                                               if x['range_1'].is_intersection(x['range_2']) 
                                               else False,axis=1)

In [158]:
promotions_df_per_week = promotions_df_per_week[promotions_df_per_week['is_on']]

In [159]:
promotions_df_per_week = promotions_df_per_week[['supplier_id','promotion_id','name','week_start','is_on']]

In [160]:
promotions_df_per_week

,supplier_id,promotion_id,name,week_start,is_on
4,1038,70baf4e3-d2fc-4fea-ae0e-0e817c260ca5,PHAN VŨ,2022-12-19,True
9,1063,35bb26c7-5519-489b-af13-4c2fe81d1401,Đại Lý Thanh Loan,2022-12-19,True
14,1322,3ffbc2ba-a21b-4bae-a1ee-2dd488117218,Đại Lý Hoàng,2022-12-19,True
19,1324,919ef3a7-9efe-469a-98c5-b4dfc036ac7d,NGỌC HÙNG,2022-12-19,True
24,1457,4583202c-0309-492d-946e-fbfacf9727d3,Hợp Tác Xã P6 Q4,2022-12-19,True
29,154,2315bcd1-5506-442c-b603-7377105fc408,Quang Vinh,2022-12-19,True
33,1783,571b8392-55b8-4c11-b7cd-e73fe8fcfa0f,Ngọc Quang Thành,2022-12-12,True
34,1783,571b8392-55b8-4c11-b7cd-e73fe8fcfa0f,Ngọc Quang Thành,2022-12-19,True
36,1783,dfb23240-648f-4785-97b2-a95c58f3f5c4,Ngọc Quang Thành,2022-11-28,True
37,1783,dfb23240-648f-4785-97b2-a95c58f3f5c4,Ngọc Quang Thành,2022-12-05,True


In [161]:
print('before: ', promotions_df_per_week.shape[0])
promotions_df_per_week.drop_duplicates().reset_index(drop=True)
print('after: ', promotions_df_per_week.shape[0])

before:  58
after:  58


In [162]:
#usamos el df de poligonos para traernos la city
promotions_df_per_week_with_cities = promotions_df_per_week.merge(supplier_cities,how='left',on='supplier_id')

In [163]:
promos_on_with_metric = promotions_df_per_week_with_cities.groupby(['supplier_id','name','city','week_start']).agg(
Active_Promotions = pd.NamedAgg(column='is_on' , aggfunc='count')).reset_index()

In [164]:
promos_on_with_metric

,supplier_id,name,city,week_start,Active_Promotions
0,1038,PHAN VŨ,Ho Chi Minh,2022-12-19,1
1,1063,Đại Lý Thanh Loan,Ho Chi Minh,2022-12-19,1
2,1322,Đại Lý Hoàng,Ho Chi Minh,2022-12-19,1
3,1324,NGỌC HÙNG,Ho Chi Minh,2022-12-19,1
4,1457,Hợp Tác Xã P6 Q4,Ho Chi Minh,2022-12-19,1
5,154,Quang Vinh,Ho Chi Minh,2022-12-19,1
6,1783,Ngọc Quang Thành,Ho Chi Minh,2022-11-28,1
7,1783,Ngọc Quang Thành,Ho Chi Minh,2022-12-05,1
8,1783,Ngọc Quang Thành,Ho Chi Minh,2022-12-12,2
9,1783,Ngọc Quang Thành,Ho Chi Minh,2022-12-19,2


In [165]:
promos_on_with_metric.rename(columns = {'name':'supplier'},inplace = True)

In [166]:
#agregamos la info al df con las otras metricas
KO_Metrics_complete = KO_Metrics_complete.merge(promos_on_with_metric, how='left',on=['supplier_id','supplier','city','week_start'])

In [167]:
KO_Metrics_complete.columns

Index(['supplier_id', 'supplier', 'city', 'week_start', 'Total_KO_Orders',
       'Total_KO_Promotion_Orders', 'GMV_KO_promotion_orders', 'GMV_KO_Prod',
       'GMV_NO_KO_Prod', 'Active_KO_Brands', 'Active_Promotions'],
      dtype='object')

In [168]:
KO_Metrics_complete = KO_Metrics_complete[['week_start','supplier_id','supplier','city','Total_KO_Orders','Total_KO_Promotion_Orders',
                        'GMV_KO_promotion_orders','GMV_KO_Prod','GMV_NO_KO_Prod','Active_KO_Brands','Active_Promotions']]

## Active Bundles KO - NO KO
##### Para esto tomamos los eans del input
##### la idea de etsa métrica es ver cuales de las bundles están asociadas a una promo activa y después contarlas

In [169]:
#levantamos las bundles del input
bundles_file = gSheets.readSheetByKey('1dcaQxt-xHG_NBuR53LGXar5GV1EtDoEd8qGXHKo6vz0')
bundles_tab = bundles_file.worksheet('bundles')
bundles_input_df = pd.DataFrame(bundles_tab.get_all_records()).reset_index(drop=True)

In [170]:
bundles_input_df.head(3)

,ean,financed
0,BDL-VN3001,Supplier
1,BDL-VN3002,Supplier
2,BDL-VN3003,Supplier


In [171]:
ean_bundles_list = set(list(bundles_input_df['ean']))

In [172]:
promos_with_bundles_query = '''
select
pi2.promotion_id, 
pi2.supplier_id,
pi2.start_date,
pi2.end_date,
pd.category,
pd.ean
from wabi2bProducts.product_details pd 
left join wabi2bPromotions.promotion_product_details ppd 
on ppd.product_display_id = pd.product_display_id 
left join wabi2bPromotions.promotion_info pi2 
on pi2.promotion_id = ppd.promotion_id
where pd.ean in '''+wabi.stringify(ean_bundles_list)+'''
and date(pi2.start_date) >= "'''+tscope_start_str+'''"
'''

In [173]:
promos_with_bundles_df_query = sql_wabilytics.querySQL(promos_with_bundles_query,'wabi2bPromotions')

In [174]:
promos_with_bundles_df = promos_with_bundles_df_query.copy(deep = True)

In [175]:
promos_with_bundles_df.head(3)

,promotion_id,supplier_id,start_date,end_date,category,ean
0,d05a6a32-d5a6-4080-a02c-8f14b88f2e24,2375,2022-12-05 17:00:00,2022-12-31 16:59:59,"[""Heineken Combo"",""Đồ uống có cồn"",""Đồ uống""]",CBVV-VNHCM12
1,086c2b77-4758-4d0b-a556-c55f06c7a7b5,225,2022-11-27 17:00:00,2022-12-05 16:59:59,"[""Heineken Combo"",""Đồ uống có cồn"",""Đồ uống""]",CBVV-VNHCM08
2,16854167-efb2-4b23-a240-192997b97940,3430,2022-12-05 17:00:00,2022-12-31 16:59:59,"[""Heineken Combo"",""Đồ uống có cồn"",""Đồ uống""]",CBVV-VNHCM05


In [176]:
promos_with_bundles_df["start_date"] = promos_with_bundles_df["start_date"].apply(lambda x: x.strftime('%Y-%m-%d'))
promos_with_bundles_df["end_date"] = promos_with_bundles_df["end_date"].apply(lambda x: x.strftime('%Y-%m-%d'))

In [177]:
promos_with_bundles_df['range_1'] =promos_with_bundles_df.apply(lambda x: DateTimeRange(pd.Timestamp(x['start_date']), pd.Timestamp(x['end_date'])),axis=1) 

In [178]:
promotions_with_bundles_per_week = promos_with_bundles_df.merge(weeks_df, how='cross')

In [179]:
promotions_with_bundles_per_week['is_on'] = promotions_with_bundles_per_week.apply(lambda x:True 
                                               if x['range_1'].is_intersection(x['range_2']) 
                                               else False,axis=1)

In [180]:
#nos quedamos solo con las promos prendidas
promotions_with_bundles_per_week = promotions_with_bundles_per_week[promotions_with_bundles_per_week['is_on']]

In [181]:
promotions_with_bundles_per_week_reduced = promotions_with_bundles_per_week[['supplier_id','promotion_id','week_start','ean']]

In [182]:
promotions_with_bundles_per_week_reduced

,supplier_id,promotion_id,week_start,ean
2,2375,d05a6a32-d5a6-4080-a02c-8f14b88f2e24,2022-12-05,CBVV-VNHCM12
3,2375,d05a6a32-d5a6-4080-a02c-8f14b88f2e24,2022-12-12,CBVV-VNHCM12
4,2375,d05a6a32-d5a6-4080-a02c-8f14b88f2e24,2022-12-19,CBVV-VNHCM12
5,225,086c2b77-4758-4d0b-a556-c55f06c7a7b5,2022-11-21,CBVV-VNHCM08
6,225,086c2b77-4758-4d0b-a556-c55f06c7a7b5,2022-11-28,CBVV-VNHCM08
...,...,...,...,...
7272,3375,7c0308a2-a6c3-4a68-a2a5-b5e7daabae25,2022-12-05,CBVV-VNHCM13
7273,3375,7c0308a2-a6c3-4a68-a2a5-b5e7daabae25,2022-12-12,CBVV-VNHCM13
7274,3375,7c0308a2-a6c3-4a68-a2a5-b5e7daabae25,2022-12-19,CBVV-VNHCM13
7276,2375,b138b84c-2b6e-4d4b-91e7-e649845a79f4,2022-11-28,CBVV-VNHCM13


In [183]:
promotions_with_bundles_per_week_reduced = promotions_with_bundles_per_week_reduced.merge(suppliers_df,how='left', on = 'supplier_id').reset_index(drop =True)
promotions_with_bundles_per_week_reduced.rename(columns= {'name':'supplier_name'}, inplace=True)

In [184]:
promotions_with_bundles_per_week_reduced

,supplier_id,promotion_id,week_start,ean,supplier_name
0,2375,d05a6a32-d5a6-4080-a02c-8f14b88f2e24,2022-12-05,CBVV-VNHCM12,Đại Lý Bia- Nước Ngọt - Sữa Thanh Huyền
1,2375,d05a6a32-d5a6-4080-a02c-8f14b88f2e24,2022-12-12,CBVV-VNHCM12,Đại Lý Bia- Nước Ngọt - Sữa Thanh Huyền
2,2375,d05a6a32-d5a6-4080-a02c-8f14b88f2e24,2022-12-19,CBVV-VNHCM12,Đại Lý Bia- Nước Ngọt - Sữa Thanh Huyền
3,225,086c2b77-4758-4d0b-a556-c55f06c7a7b5,2022-11-21,CBVV-VNHCM08,HOA ĐÀO
4,225,086c2b77-4758-4d0b-a556-c55f06c7a7b5,2022-11-28,CBVV-VNHCM08,HOA ĐÀO
...,...,...,...,...,...
3364,3375,7c0308a2-a6c3-4a68-a2a5-b5e7daabae25,2022-12-05,CBVV-VNHCM13,Đại Lý Anh Thơ
3365,3375,7c0308a2-a6c3-4a68-a2a5-b5e7daabae25,2022-12-12,CBVV-VNHCM13,Đại Lý Anh Thơ
3366,3375,7c0308a2-a6c3-4a68-a2a5-b5e7daabae25,2022-12-19,CBVV-VNHCM13,Đại Lý Anh Thơ
3367,2375,b138b84c-2b6e-4d4b-91e7-e649845a79f4,2022-11-28,CBVV-VNHCM13,Đại Lý Bia- Nước Ngọt - Sữa Thanh Huyền


In [185]:
print('before: ', promotions_with_bundles_per_week_reduced.shape[0])
promotions_with_bundles_per_week_reduced.drop_duplicates().reset_index(drop=True)
print('after: ', promotions_with_bundles_per_week_reduced.shape[0])

before:  3369
after:  3369


In [186]:
#usamos el df de poligonos para traernos la city
promotions_with_bundles_per_week_with_cities = promotions_with_bundles_per_week_reduced.merge(supplier_cities,how='left',on='supplier_id')

In [187]:
promotions_with_bundles_per_week_with_cities

,supplier_id,promotion_id,week_start,ean,supplier_name,country_id,city
0,2375,d05a6a32-d5a6-4080-a02c-8f14b88f2e24,2022-12-05,CBVV-VNHCM12,Đại Lý Bia- Nước Ngọt - Sữa Thanh Huyền,vn,Ho Chi Minh
1,2375,d05a6a32-d5a6-4080-a02c-8f14b88f2e24,2022-12-12,CBVV-VNHCM12,Đại Lý Bia- Nước Ngọt - Sữa Thanh Huyền,vn,Ho Chi Minh
2,2375,d05a6a32-d5a6-4080-a02c-8f14b88f2e24,2022-12-19,CBVV-VNHCM12,Đại Lý Bia- Nước Ngọt - Sữa Thanh Huyền,vn,Ho Chi Minh
3,225,086c2b77-4758-4d0b-a556-c55f06c7a7b5,2022-11-21,CBVV-VNHCM08,HOA ĐÀO,vn,Ho Chi Minh
4,225,086c2b77-4758-4d0b-a556-c55f06c7a7b5,2022-11-28,CBVV-VNHCM08,HOA ĐÀO,vn,Ho Chi Minh
...,...,...,...,...,...,...,...
3364,3375,7c0308a2-a6c3-4a68-a2a5-b5e7daabae25,2022-12-05,CBVV-VNHCM13,Đại Lý Anh Thơ,vn,Ho Chi Minh
3365,3375,7c0308a2-a6c3-4a68-a2a5-b5e7daabae25,2022-12-12,CBVV-VNHCM13,Đại Lý Anh Thơ,vn,Ho Chi Minh
3366,3375,7c0308a2-a6c3-4a68-a2a5-b5e7daabae25,2022-12-19,CBVV-VNHCM13,Đại Lý Anh Thơ,vn,Ho Chi Minh
3367,2375,b138b84c-2b6e-4d4b-91e7-e649845a79f4,2022-11-28,CBVV-VNHCM13,Đại Lý Bia- Nước Ngọt - Sữa Thanh Huyền,vn,Ho Chi Minh


In [188]:
#agregamos el financed del input
bundles_promos = promotions_with_bundles_per_week_with_cities.merge(bundles_input_df,how = 'left', on = 'ean')

In [189]:
bundles_promos['KO_financed'] = bundles_promos.apply(lambda x: 1 if x['financed']== 'Coca Cola' else 0,axis = 1)

In [190]:
bundles_promos['wabi_financed'] = bundles_promos.apply(lambda x: 1 if x['financed']== 'Supplier' else 0,axis = 1)

In [191]:
bundles_promos[(bundles_promos['supplier_id']=='1038') & (bundles_promos['city']=='Ho Chi Minh') &
              (bundles_promos['week_start']=='2022-11-21')]

,supplier_id,promotion_id,week_start,ean,supplier_name,country_id,city,financed,KO_financed,wabi_financed


In [192]:
bundles_promos_metric = bundles_promos.groupby(['supplier_id','supplier_name','city','week_start']).agg(
active_bundle_ko = pd.NamedAgg(column='KO_financed' , aggfunc='sum'),
active_bundle_wabi = pd.NamedAgg(column='wabi_financed' , aggfunc='sum')).reset_index()

In [193]:
bundles_promos_metric.rename(columns= {'supplier_name':'supplier'}, inplace=True)

In [194]:
bundles_promos_metric.head(3)

,supplier_id,supplier,city,week_start,active_bundle_ko,active_bundle_wabi
0,1033,PHƯỚC DŨNG,Ho Chi Minh,2022-12-19,5,0
1,1038,PHAN VŨ,Ho Chi Minh,2022-12-05,20,0
2,1038,PHAN VŨ,Ho Chi Minh,2022-12-12,32,0


In [195]:
bundles_promos_metric[bundles_promos_metric['active_bundle_wabi']!=0]

,supplier_id,supplier,city,week_start,active_bundle_ko,active_bundle_wabi


In [196]:
KO_Metrics_complete

,week_start,supplier_id,supplier,city,Total_KO_Orders,Total_KO_Promotion_Orders,GMV_KO_promotion_orders,GMV_KO_Prod,GMV_NO_KO_Prod,Active_KO_Brands,Active_Promotions
0,2022-11-21,1038,PHAN VŨ,Ho Chi Minh,116,0,0,72842000.0,795338180.0,3.0,NaN
1,2022-11-28,1038,PHAN VŨ,Ho Chi Minh,122,1,9623400.00,83391500.0,688413404.0,3.0,NaN
2,2022-12-05,1038,PHAN VŨ,Ho Chi Minh,135,0,0,85516500.0,708351620.0,3.0,NaN
3,2022-12-12,1038,PHAN VŨ,Ho Chi Minh,112,1,5830960.00,71437000.0,760856140.0,3.0,NaN
4,2022-12-19,1038,PHAN VŨ,Ho Chi Minh,32,1,5584960.00,32685500.0,150566580.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
238,2022-12-05,892,Đại lý Long Khuê- Thường Tín,Hanoi,12,0,0,14352000.0,91555000.0,1.0,NaN
239,2022-12-12,892,Đại lý Long Khuê- Thường Tín,Hanoi,11,0,0,19688000.0,331875000.0,2.0,NaN
240,2022-12-19,892,Đại lý Long Khuê- Thường Tín,Hanoi,1,0,0,920000.0,NaN,4.0,NaN
241,2022-11-28,968,Đại lý Thùy Trang- Thanh Trì,Hanoi,1,0,0,177000.0,2079000.0,NaN,NaN


In [197]:
KO_Metrics_complete = KO_Metrics_complete.merge(bundles_promos_metric, how='left',on=['supplier_id','supplier','city','week_start'])

In [198]:
KO_Metrics_complete.head(3)

,week_start,supplier_id,supplier,city,Total_KO_Orders,Total_KO_Promotion_Orders,GMV_KO_promotion_orders,GMV_KO_Prod,GMV_NO_KO_Prod,Active_KO_Brands,Active_Promotions,active_bundle_ko,active_bundle_wabi
0,2022-11-21,1038,PHAN VŨ,Ho Chi Minh,116,0,0,72842000.0,795338180.0,3.0,NaN,NaN,NaN
1,2022-11-28,1038,PHAN VŨ,Ho Chi Minh,122,1,9623400.00,83391500.0,688413404.0,3.0,NaN,NaN,NaN
2,2022-12-05,1038,PHAN VŨ,Ho Chi Minh,135,0,0,85516500.0,708351620.0,3.0,NaN,20.0,0.0


# daily_ko_prices Tab
#### - Tomamos solamente las ordenes que fueron shippeadas
#### - Tomamos solamente las ordenes de los suppliers del input
#### - Tomamos solamente las ordenes con los eans del input

# Selected Eans

In [199]:
eans_file = gSheets.readSheetByKey('1dcaQxt-xHG_NBuR53LGXar5GV1EtDoEd8qGXHKo6vz0')
eans_tab = eans_file.worksheet('eans')
eans_df = pd.DataFrame(eans_tab.get_all_records()).reset_index(drop=True)

In [200]:
eans_list = list(set(eans_df["ean"].astype(str)))

In [201]:
#nos quedamos con las ordenes que vienen del global para que sea lineal con lo que se hizo arriba
#hasta acá ya tenemos filtradas los ordenes por los supplier_id del input, por scope y quedandonos con las
#ordenes que fueron shippeadas
daily_orders =  global_data0_reduced.copy(deep = True)

In [202]:
daily_orders.head(3)

,supplier_order_id,order_id,supplier_id,status,supplier,created,confirmed_at,canceled_by,supplier_cancellation_reason,customer_cancellation_reason,total,discount,city,country_id,created_date,shippedAt,year,week,month,week_start
253687,2674050,2482103,340,SHIPPED,Toàn Trường Thịnh,2022-11-21 00:33:05,2022-11-21 00:37:10,None,None,None,2205000.0,33074.84,Ho Chi Minh,vn,2022-11-21,2022-11-21 16:02:17,2022,47,11,2022-11-21
253688,2674073,2482122,3375,SHIPPED,Đại Lý Anh Thơ,2022-11-21 01:08:36,2022-11-21 01:46:44,None,None,None,6920000.0,102195.00,Ho Chi Minh,vn,2022-11-21,2022-11-21 19:29:47,2022,47,11,2022-11-21
253689,2674080,2482128,2097,SHIPPED,Nghĩa Phượng,2022-11-21 01:24:40,2022-11-20 22:49:11,None,None,None,6375000.0,95625.00,Da Nang,vn,2022-11-21,2022-11-21 18:03:47,2022,47,11,2022-11-21


In [203]:
sup_order_id_list = list(set(daily_orders['supplier_order_id'].astype(str)))

In [204]:
orders_query = '''
SELECT 
	soi.supplier_order_id,
	soi.amount,
	soi.ean,
    soi.name,
	soi.supplier_id,
	pd_aux.manufacturer,
	soi.price,
    soi.units,
    soi.brand,
    pd_aux.product_display_id
FROM wabi2bOrders.supplier_order_items soi
inner JOIN (SELECT pd.product_id, manufacturer, pd.product_display_id 
            FROM wabi2bProducts.product_details pd
            GROUP BY pd.product_id) pd_aux on pd_aux.product_id = soi.product_id 
WHERE soi.supplier_order_id IN '''+wabi.stringify(sup_order_id_list)+''';
'''


In [205]:
orders_df_query = sql_wabilytics.querySQL(orders_query,'wabi2bOrders')

In [206]:
orders_df = orders_df_query.copy(deep = True)

In [207]:
orders_df.head(3)

,supplier_order_id,amount,ean,name,supplier_id,manufacturer,price,units,brand,product_display_id
0,2778439,408000.0,8936010688142,Thùng Phở Gà Cung Đình Nước Cốt Xương Hầm 30 G...,1089,Afotech,204000.0,1,Micoem,204478
1,2811063,770000.0,1893503950070,Thùng Bột Ngọt Ajinomoto 12 Gói x 1KG,2629,Ajinomoto,770000.0,1,Ajinomoto,206245
2,2704524,5120000.0,18935012413335,Thùng Bia Sài Gòn 333 Mẫu Trúng Thưởng 2022 24...,635,Sabeco Brewery,256000.0,1,Sabeco,212712


In [208]:
orders_df = orders_df.merge(daily_orders[['supplier_order_id','created_date']], how='inner', on ='supplier_order_id')

In [209]:
#filtramos por los eans que nos interesan
orders_df_reduced = orders_df.loc[orders_df['ean'].isin(eans_list)].reset_index(drop=True)

In [210]:
print('before: ', orders_df_reduced.shape[0])
orders_df_reduced = orders_df_reduced.drop_duplicates().reset_index(drop=True)
print('after: ', orders_df_reduced.shape[0])

before:  32479
after:  32479


In [211]:
orders_df_reduced['manufacturer'].unique()

array(['Nestlé', 'Masan Group', 'Coke Marketplace', 'TCP', 'Acecook',
       'Vinamilk', 'Habeco', 'Sabeco Brewery', 'Heineken Company',
       'CALOFIC', 'Tân Hiệp Phát', 'PepsiCo', 'URC Vietnam',
       'Khanh Hoa Sanest Soft Drink JSC', 'FrieslandCampina',
       'Carlsberg Việt Nam', 'Ajinomoto', 'Bundles - Vn'], dtype=object)

In [212]:
orders_df_reduced = orders_df_reduced.merge(supplier_cities,how='left',on='supplier_id')

In [213]:
orders_df_reduced

,supplier_order_id,amount,ean,name,supplier_id,manufacturer,price,units,brand,product_display_id,created_date,country_id,city
0,2778439,672000.0,8934804025766,Thùng Nestle Milo hộp giấy 180ml,1089,Nestlé,336000.0,1,Milo,66405,2022-12-07,vn,Hanoi
1,2811063,1005000.0,8934804025766,Thùng Nestle Milo hộp giấy 180ml,2629,Nestlé,335000.0,48,Milo,66405,2022-12-13,vn,Hanoi
2,2811063,636000.0,8936017368104,Thùng mì Omachi vị sườn hầm ngũ quả gói 80g x30,2629,Masan Group,212000.0,30,Omachi,66476,2022-12-13,vn,Hanoi
3,2811063,354000.0,18935049501509,Thùng Sữa Trái Cây Nutriboost Hương Cam 24 Cha...,2629,Coke Marketplace,177000.0,1,Nutriboost,225692,2022-12-13,vn,Hanoi
4,2811063,1225000.0,8850228000151,Thùng Redbull Thái Lon 250 ml,2629,TCP,245000.0,1,RedBull,66502,2022-12-13,vn,Hanoi
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32474,2701049,4129200.0,8936007080016,Thùng nước yến sào Khánh Hòa Sanest lon 190ml x30,2406,Khanh Hoa Sanest Soft Drink JSC,229400.0,30,Sanest,125176,2022-11-24,vn,Da Nang
32475,2755547,2730000.0,8938508429128,Thùng bia Red Ruby lon 330ml x24,1324,Masan Group,182000.0,1,Red Ruby,134520,2022-12-03,vn,Ho Chi Minh
32476,2838190,905000.0,8938508429128,Thùng bia Red Ruby lon 330ml x24,2496,Masan Group,181000.0,1,Red Ruby,134520,2022-12-18,vn,Ho Chi Minh
32477,2814013,3620000.0,8938508429128,Thùng bia Red Ruby lon 330ml x24,2496,Masan Group,181000.0,1,Red Ruby,134520,2022-12-14,vn,Ho Chi Minh


In [214]:
#tomo el amount para hacer el GMV ya que es el unit price* cantidad vendida de ese producto
#como el price es el mismo para ese dia y ese supplier en particular, tomo la primera aparición
final_daily_metrics = orders_df_reduced.groupby(['created_date','supplier_id','city','ean','name','units','manufacturer','brand']).agg(
GMV = pd.NamedAgg(column='amount' , aggfunc='sum'),
unit_price = pd.NamedAgg(column='price' , aggfunc='first')
).reset_index()

In [215]:
final_daily_metrics.rename(columns= {'created_date':'day','name':'product_name'}, inplace=True)

In [216]:
final_daily_metrics

,day,supplier_id,city,ean,product_name,units,manufacturer,brand,GMV,unit_price
0,2022-11-21,1038,Ho Chi Minh,18934673573341,Thùng Sữa Hộp Vinamilk Có Đường Mẫu Mới 4...,1,Vinamilk,Vinamilk,8125000.0,325000.0
1,2022-11-21,1038,Ho Chi Minh,18935049501509,Thùng Sữa Trái Cây Nutriboost Hương Cam 24 Cha...,1,Coke Marketplace,Nutriboost,676000.0,169000.0
2,2022-11-21,1038,Ho Chi Minh,18935049501623,Thùng Sữa Trái Cây Nutriboost Hương Dâu 24 Cha...,1,Coke Marketplace,Nutriboost,1183000.0,169000.0
3,2022-11-21,1038,Ho Chi Minh,18935049502670,Thùng Nước Ngọt Coca Cola 24 Chai x 390ML,1,Coke Marketplace,Coca-Cola,2242000.0,118000.0
4,2022-11-21,1038,Ho Chi Minh,18936193070072,Thùng Tăng Lực Number One 24 Chai x 330ML,1,Tân Hiệp Phát,NumberOne,6020000.0,172000.0
...,...,...,...,...,...,...,...,...,...,...
13606,2022-12-21,1359,Da Nang,8934673132343,Thùng Sữa bịch Vinamilk 100% ít đường 220ml,48,Vinamilk,Vinamilk,600000.0,300000.0
13607,2022-12-21,1359,Da Nang,8934673581349,Thùng 48 hộp sữa Vinamilk 100% 180ml ít đường,1,Vinamilk,Vinamilk,648000.0,324000.0
13608,2022-12-21,2785,Hanoi,8934563305048,Thùng Mì Hảo Hảo Tôm Chua Cay 30 Gói x 75G,1,Acecook,Hao Hao,2340000.0,117000.0
13609,2022-12-21,2785,Hanoi,8935039500592,Bịch Ajinomoto 454g,1,Ajinomoto,Ajinomoto,1239500.0,1239500.0


In [217]:
final_daily_metrics = final_daily_metrics.merge(suppliers_df,how='left', on = 'supplier_id').reset_index(drop =True)
final_daily_metrics.rename(columns= {'name':'supplier_name'}, inplace=True)

In [218]:
final_daily_metrics

,day,supplier_id,city,ean,product_name,units,manufacturer,brand,GMV,unit_price,supplier_name
0,2022-11-21,1038,Ho Chi Minh,18934673573341,Thùng Sữa Hộp Vinamilk Có Đường Mẫu Mới 4...,1,Vinamilk,Vinamilk,8125000.0,325000.0,PHAN VŨ
1,2022-11-21,1038,Ho Chi Minh,18935049501509,Thùng Sữa Trái Cây Nutriboost Hương Cam 24 Cha...,1,Coke Marketplace,Nutriboost,676000.0,169000.0,PHAN VŨ
2,2022-11-21,1038,Ho Chi Minh,18935049501623,Thùng Sữa Trái Cây Nutriboost Hương Dâu 24 Cha...,1,Coke Marketplace,Nutriboost,1183000.0,169000.0,PHAN VŨ
3,2022-11-21,1038,Ho Chi Minh,18935049502670,Thùng Nước Ngọt Coca Cola 24 Chai x 390ML,1,Coke Marketplace,Coca-Cola,2242000.0,118000.0,PHAN VŨ
4,2022-11-21,1038,Ho Chi Minh,18936193070072,Thùng Tăng Lực Number One 24 Chai x 330ML,1,Tân Hiệp Phát,NumberOne,6020000.0,172000.0,PHAN VŨ
...,...,...,...,...,...,...,...,...,...,...,...
13606,2022-12-21,1359,Da Nang,8934673132343,Thùng Sữa bịch Vinamilk 100% ít đường 220ml,48,Vinamilk,Vinamilk,600000.0,300000.0,Đại Lý Duy Vinh
13607,2022-12-21,1359,Da Nang,8934673581349,Thùng 48 hộp sữa Vinamilk 100% 180ml ít đường,1,Vinamilk,Vinamilk,648000.0,324000.0,Đại Lý Duy Vinh
13608,2022-12-21,2785,Hanoi,8934563305048,Thùng Mì Hảo Hảo Tôm Chua Cay 30 Gói x 75G,1,Acecook,Hao Hao,2340000.0,117000.0,Đại lý Anh Quân- Thường Tín
13609,2022-12-21,2785,Hanoi,8935039500592,Bịch Ajinomoto 454g,1,Ajinomoto,Ajinomoto,1239500.0,1239500.0,Đại lý Anh Quân- Thường Tín


In [219]:
final_daily_metrics= final_daily_metrics[['day','supplier_id','supplier_name','city','ean','product_name','units','manufacturer','brand','GMV','unit_price']]

In [220]:
final_daily_metrics.head(3)

,day,supplier_id,supplier_name,city,ean,product_name,units,manufacturer,brand,GMV,unit_price
0,2022-11-21,1038,PHAN VŨ,Ho Chi Minh,18934673573341,Thùng Sữa Hộp Vinamilk Có Đường Mẫu Mới 4...,1,Vinamilk,Vinamilk,8125000.0,325000.0
1,2022-11-21,1038,PHAN VŨ,Ho Chi Minh,18935049501509,Thùng Sữa Trái Cây Nutriboost Hương Cam 24 Cha...,1,Coke Marketplace,Nutriboost,676000.0,169000.0
2,2022-11-21,1038,PHAN VŨ,Ho Chi Minh,18935049501623,Thùng Sữa Trái Cây Nutriboost Hương Dâu 24 Cha...,1,Coke Marketplace,Nutriboost,1183000.0,169000.0


In [221]:
final_daily_metrics["manufacturer"] = final_daily_metrics["manufacturer"].apply(lambda x: x.lower())

In [222]:
aditional_info_manufacturers_list = list(set(final_daily_metrics["manufacturer"].astype(str)))

In [223]:
coca_aditional_info_manufacturers_list = KO_list(aditional_info_manufacturers_list)

In [224]:
final_daily_metrics['KO_products'] = final_daily_metrics.apply(lambda x: 'KO' if x['manufacturer'] in coca_aditional_info_manufacturers_list else 'NO_KO',axis = 1)

In [225]:
final_daily_metrics

,day,supplier_id,supplier_name,city,ean,product_name,units,manufacturer,brand,GMV,unit_price,KO_products
0,2022-11-21,1038,PHAN VŨ,Ho Chi Minh,18934673573341,Thùng Sữa Hộp Vinamilk Có Đường Mẫu Mới 4...,1,vinamilk,Vinamilk,8125000.0,325000.0,NO_KO
1,2022-11-21,1038,PHAN VŨ,Ho Chi Minh,18935049501509,Thùng Sữa Trái Cây Nutriboost Hương Cam 24 Cha...,1,coke marketplace,Nutriboost,676000.0,169000.0,KO
2,2022-11-21,1038,PHAN VŨ,Ho Chi Minh,18935049501623,Thùng Sữa Trái Cây Nutriboost Hương Dâu 24 Cha...,1,coke marketplace,Nutriboost,1183000.0,169000.0,KO
3,2022-11-21,1038,PHAN VŨ,Ho Chi Minh,18935049502670,Thùng Nước Ngọt Coca Cola 24 Chai x 390ML,1,coke marketplace,Coca-Cola,2242000.0,118000.0,KO
4,2022-11-21,1038,PHAN VŨ,Ho Chi Minh,18936193070072,Thùng Tăng Lực Number One 24 Chai x 330ML,1,tân hiệp phát,NumberOne,6020000.0,172000.0,NO_KO
...,...,...,...,...,...,...,...,...,...,...,...,...
13606,2022-12-21,1359,Đại Lý Duy Vinh,Da Nang,8934673132343,Thùng Sữa bịch Vinamilk 100% ít đường 220ml,48,vinamilk,Vinamilk,600000.0,300000.0,NO_KO
13607,2022-12-21,1359,Đại Lý Duy Vinh,Da Nang,8934673581349,Thùng 48 hộp sữa Vinamilk 100% 180ml ít đường,1,vinamilk,Vinamilk,648000.0,324000.0,NO_KO
13608,2022-12-21,2785,Đại lý Anh Quân- Thường Tín,Hanoi,8934563305048,Thùng Mì Hảo Hảo Tôm Chua Cay 30 Gói x 75G,1,acecook,Hao Hao,2340000.0,117000.0,NO_KO
13609,2022-12-21,2785,Đại lý Anh Quân- Thường Tín,Hanoi,8935039500592,Bịch Ajinomoto 454g,1,ajinomoto,Ajinomoto,1239500.0,1239500.0,NO_KO


# Pegamos cositas en el sheet

#### daily_ko_prices Tab

In [226]:
#me armo una lista con todos los días que hay desde el scope_start y el scope_end
date_range = DateTimeRange(tscope_start, tscope_end)
date_range_list = []
for value in date_range.range(datetime.timedelta(days=1)):
    date_range_list.append(value.strftime('%Y-%m-%d'))

In [227]:
file = gSheets.readSheetByKey('18ODZw3RpJyEEG1MJIbqdxD0leGbRj6XFr7wlQyUJFiY')
sheet_tab = file.worksheet('daily_ko_prices')
df = pd.DataFrame(sheet_tab.get_all_records()).reset_index(drop=True)

In [228]:
updated_df = final_daily_metrics
if len(df) > 0:
    #acá volamos todos los registros que tengan fecha entre [scope_start,scope_end]
    df_without_4_weeks = df.loc[~df['day'].isin(date_range_list)].reset_index(drop=True)
    updated_df = updated_df.append(df_without_4_weeks)
updated_df.sort_values(by =["day", "supplier_id"], inplace = True)
sheet_tab.clear()
set_with_dataframe(sheet_tab, updated_df)

C:\Users\camilafernandez_wabi\AppData\Local\Temp\ipykernel_10844\3050912537.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  updated_df = updated_df.append(df_without_4_weeks)


#### weekly_supplier_ko_prod Tab

In [229]:
file = gSheets.readSheetByKey('18ODZw3RpJyEEG1MJIbqdxD0leGbRj6XFr7wlQyUJFiY')
sheet_tab = file.worksheet('weekly_supplier_ko_prod')
df = pd.DataFrame(sheet_tab.get_all_records()).reset_index(drop=True)

In [230]:
updated_df = KO_Metrics_complete
if len(df) > 0:
    df_without_5_weeks = df.loc[~df['week_start'].isin(weeks_list)].reset_index(drop=True)
    #filtro y me quedo con las semanas menores a las 5 semanas calculadas
    updated_df = updated_df.append(df_without_4_weeks)
updated_df.sort_values(by =["week_start", "supplier_id"], inplace = True)
sheet_tab.clear()
set_with_dataframe(sheet_tab, updated_df)

C:\Users\camilafernandez_wabi\AppData\Local\Temp\ipykernel_10844\2112424247.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  updated_df = updated_df.append(df_without_4_weeks)
